# Global Parameters & Imports for below functions:

In [ ]:
import requests
import urllib.request
import json
import html
import re
from haversine import haversine, haversine_vector
import heapq
import pandas as pd
from dateutil import parser

limit = 1
radius = 6
lat = 52.635875 #uk - norfolk
lng = 1.301 #uk - norfolk
# lat = 52.50003299 #germany - berlin
# lng = 13.3913285 #germany - berlin

# Notes about the Overall Design Schema adopted in the POI Scraper Project:
A consistent design schema was adopted in this project with the aim of making all code as similar and safe as possible, allowing easy understanding of the projects inner workings for newcomers, while making altering prior code easier. This Schema is detailed below:
## New Heap code structure that deals with all cases of missing and out of order days:
* This is the standardised structure, introduced in **v9**, used throughout all store functions that use an "integer based" system for keeping store opening hours data. This term encompassesmeans all systems where the opening hours are not stored in a dictionary with the keys explicitly identifying/being the days themselves. For example any system where the days are stored as an array would count as an "integer based" system, or one where the days were stored in a dictionary with the keys being 0-n with the day then stored as a field inside this dictionary (e.g. Co-op, or slightly less obviously, Sainsbury's). However a system such as Tesco's where the keys of the dictionary are the days themselves, a heap system is unneccessary and therefore a simple loop containing a try, except clause suffices (these such systems are not counter as "integer based" systems). At the core of this structure, the code for processing and producing the dictionaries that form the opening hours array is placed.

In [1]:
# Copyable Heap structure code:
dayDoneSet = set()
daysHeap = []
for dayObject in openingHours:
        day = intermediaryDayKeys[dayObject["day"]]
        #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
        if day not in dayDoneSet:
            dayDoneSet.add(day)
            #keyHours processing code
            #INSERT HOUR PROCESSING CODE HERE
        
    

            heapq.heappush(daysHeap, [day,keyHours]) #(REPLACE hoursArray.append(keyHours) with this statement)
#Check for any missing days and ensure all days are in order for insertion into our database
checkedUpToDay = -1
for day in heapq.nsmallest(7, daysHeap):
    checkedUpToDay += 1
    while day[0] > checkedUpToDay:
        closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                'open' : False  }
        heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
        checkedUpToDay += 1
while len(daysHeap) < 7:
    if len(daysHeap) > 0:
        checkedUpToDay = daysHeap[-1][0] + 1
    else:
        # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
        checkedUpToDay = 0
    closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                'open' : False  }
    heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
if len(daysHeap) != 7:
    return False
hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
return hoursArray

NameError: name 'openingHours' is not defined

## Additional notes about design schema throughout:
In addition to that stated above, in an API system that returns (or alternatively is able to return) all stores rather than the stores close to a pair of coordinates, the standardised approach adopted in this project has been to set up a database of preprepared opening hours arrays, stored along with the coordinates of the stores themselves. This database is then called by the get_X_data(lat, lng) function, where the haversine_vector function is called to efficiently compute the distances between all stores and the desired point. The minimum index is then retrieved and the opening hours of that indexed store are returned (see EDEKA or Netto's Sections in the below document for examples of both parts of this process).

# Documentation for all Stores:

# REWE's API (Germany)
## Base URL:
https://www.rewe.de/market/content/marketsearch
## Parameters:
**Note:** Parameters are not usually supplied in a traditional fashion, they are supplied as "Form Data" (under the Chrome network inspector). However, after testing, the API does seem to return results if the same parameters are supplied as traditional queries?


**WARNING:** If the API call is made from the python requests module, it will return 403 Forbidden as its status code. After much work it was discovered that a GET request made through the urllib module using the headers shown below (specifically "user-agent") is succesful (200 OK status code).
* searchString: Literally searches the store names and addresses for **STRING MATCH** so if post code is 12099 giving 12098 will give no results for example.
* **unknown** latitude and longitude option?? Tried: lat lng, lat lon, latitude longitude, coordinates. I believe their system is extremely simplistic and has no form of search beyond searchString, or their premade city tabs.
* page = 0 (which batch of pageSize stores to return)
* city (optional)
* pageSize = number of stores to return (up to 500, beyond there only 500 stores will be returned, and page system breaks(?))


## Sample API Call:
https://www.rewe.de/market/content/marketsearch?searchString=REWE&pageSize=500&page=0
## Sample API Call Python Code:

In [1]:
import urllib.request
import json
API_URL = "https://www.rewe.de/market/content/marketsearch"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-language": "en-US,en;q=0.9",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1"
}
params = {  'searchString' : "REWE",
            'pageSize' : 500,
            'page' : 0}
req_url = API_URL + '?' + '&'.join(k + '=' + str(v) for k, v in params.items())
resp = urllib.request.urlopen(urllib.request.Request(url=req_url, data=None, headers=headers))
data = resp.read().decode('utf-8')
res = json.loads(data)

## Sample Response:
**Note:** Two sample responses are expanded (the second one partially) in order to exacerbate that stores may have multiple "opening hours day ranges", and thus this must be accounted for.

**WARNING:** Occasionally some stores return bugged store opening hours (always the same stores) where the day ranges overlap and often disagree for hours of individual days. 
**E.g.**:
{'days': 'Do', 'hours': '08:00 - 22:00'}, {'days': 'Fr-Sa', 'hours': '08:00 - 20:00'}, {'days': 'Mo-Sa', 'hours': '08:00 - 20:00'}
This is why the dayDoneSet were introduced in the set_up_rewe_database() function.

**WARNING:** Some stores return empty lists for opening hours, there is a comment in the code where this check is performed, any code dealing with these stores e.g. flagging them as closed may be performed there.

In [1]:
markets: [{marketManager: "", phone: null, advertisingCounty: null, regionShort: "WE",…},…]
    [0 … 99]
        0: {marketManager: "", phone: null, advertisingCounty: null, regionShort: "WE",…}
            address: {street: "Saarwerdenstraße", houseNumber: "6", postalCode: "41541", city: "Dormagen - Zons",…}
                city: "Dormagen - Zons"
                houseNumber: "6"
                postalCode: "41541"
                state: "Nordrhein-Westfalen"
                street: "Saarwerdenstraße"
                streetWithNumber: "Saarwerdenstraße 6"
            advertisingCounty: null
            closedFrom: null
            closedUntil: null
            company: {name: "REWE Bernd Uderhardt oHG", street: "Saarwerdenstr. 6", zipCode: "41541", city: "Dormagen-Zons"}
                city: "Dormagen-Zons"
                name: "REWE Bernd Uderhardt oHG"
                street: "Saarwerdenstr. 6"
                zipCode: "41541"
            companyName: "REWE Bernd Uderhardt oHG"
            dortmund: false
            externalLink: null
            firstOpeningDate: "2004-12-01"
            geoLocation: {latitude: 51.123861, longitude: 6.84568}
                latitude: 51.123861
                longitude: 6.84568
            holiday: null
            holidayDate: null
            id: 1765235
            marketManager: ""
            nahkauf: false
            openOnSunday: false
            opened: true
            openedUntil: {hour: 22, minute: 0, second: 0, nano: 0}
                hour: 22
                minute: 0
                nano: 0
                second: 0
            openingHours: {condensed: [{days: "Mo-Sa", hours: "07:00 - 22:00"}],…}
                condensed: [{days: "Mo-Sa", hours: "07:00 - 22:00"}]
                    0: {days: "Mo-Sa", hours: "07:00 - 22:00"}
                        days: "Mo-Sa"
                        hours: "07:00 - 22:00"
                dayAndTimeRanges: [{startDay: "MONDAY", endDay: "SATURDAY", startTime: {hour: 7, minute: 0, second: 0, nano: 0},…}]
                    0: {startDay: "MONDAY", endDay: "SATURDAY", startTime: {hour: 7, minute: 0, second: 0, nano: 0},…}
                        closes: "22:00"
                        endDay: "SATURDAY"
                        endTime: {hour: 22, minute: 0, second: 0, nano: 0}
                            hour: 22
                            minute: 0
                            nano: 0
                            second: 0
                        hour: 22
                        minute: 0
                        nano: 0
                        second: 0
                        opens: "07:00"
                        seoRepresentation: "Mo-Sa 07:00-22:00"
                        startDay: "MONDAY"
                        startTime: {hour: 7, minute: 0, second: 0, nano: 0}
                            hour: 7
                            minute: 0
                            nano: 0
                            second: 0
            phone: null
            pickupMarket: true
            regionShort: "WE"
            specialEnd: null
            specialOpening: null
            specialOpeningHours: null
            specialStart: null
            state: "Nordrhein-Westfalen"
            stateShort: "NW"
            temporaryClosed: false
            type: {name: "REWE", id: null}
            id: null
            name: "REWE"
            wawi: "43655399"
        1: {marketManager: "", phone: "02631-946281", advertisingCounty: null, regionShort: "WE",…}
        2: {marketManager: "", phone: "089-30000521", advertisingCounty: null, regionShort: "SU",…}
        3: {marketManager: "", phone: "0211-86283696", advertisingCounty: null, regionShort: "WE",…}
        4: {marketManager: "", phone: "08331-9851762", advertisingCounty: null, regionShort: "SU",…}
        5: {marketManager: "", phone: "0721-83185594", advertisingCounty: null, regionShort: "SW",…}
        6: {marketManager: "", phone: "06442-95190", advertisingCounty: null, regionShort: "MI",…}
        7: {marketManager: "", phone: "0531-2502474", advertisingCounty: null, regionShort: "NO",…}
        8: {marketManager: "", phone: "07072-6009502", advertisingCounty: null, regionShort: "SW",…}
        9: {marketManager: "", phone: "0681-98229365", advertisingCounty: null, regionShort: "SW",…}
        10: {marketManager: "", phone: "04151-866759", advertisingCounty: null, regionShort: "NO",…}
        11: {marketManager: "", phone: "0821-52138832", advertisingCounty: null, regionShort: "SU",…}
        12: {marketManager: "", phone: "0221-9359560", advertisingCounty: null, regionShort: "WE",…}
        13: {marketManager: "", phone: "036020-769723", advertisingCounty: null, regionShort: "OS",…}
        14: {marketManager: "", phone: "07031-744914", advertisingCounty: null, regionShort: "SW",…}
        15: {marketManager: "", phone: "06032-93760", advertisingCounty: null, regionShort: "MI",…}
        16: {marketManager: "", phone: "08094-907956", advertisingCounty: null, regionShort: "SU",…}
        17: {marketManager: "", phone: "02401/896400", advertisingCounty: null, regionShort: "WE",…}
        18: {marketManager: "", phone: "02131-1249939", advertisingCounty: null, regionShort: "WE",…}
        19: {marketManager: "", phone: "09972-9019010", advertisingCounty: null, regionShort: "SU",…}
        20: {marketManager: "", phone: "08841-3022", advertisingCounty: null, regionShort: "SU",…}
        21: {marketManager: "", phone: "089-96209706", advertisingCounty: null, regionShort: "SU",…}
        22: {marketManager: "", phone: "0821-6084537", advertisingCounty: null, regionShort: "SU",…}
        23: {marketManager: "", phone: "02452-98872-0", advertisingCounty: null, regionShort: "WE",…}
        24: {marketManager: "", phone: "069-90749571", advertisingCounty: null, regionShort: "MI",…}
        25: {marketManager: "", phone: "06092-97250", advertisingCounty: null, regionShort: "MI",…}
        26: {marketManager: "", phone: "0214-8404797", advertisingCounty: null, regionShort: "WE",…}
        27: {marketManager: "", phone: "02722-9261-0", advertisingCounty: null, regionShort: "WE",…}
        28: {marketManager: "", phone: "06131-6278995", advertisingCounty: null, regionShort: "MI",…}
        29: {marketManager: "", phone: "0511-804201", advertisingCounty: null, regionShort: "NO",…}
        30: {marketManager: "", phone: "08142-29962", advertisingCounty: null, regionShort: "SU",…}
        31: {marketManager: "", phone: "0911-97901923", advertisingCounty: null, regionShort: "SU",…}
        32: {marketManager: "", phone: "03928-7699822", advertisingCounty: null, regionShort: "OS",…}
        33: {marketManager: "", phone: "089-89398183", advertisingCounty: null, regionShort: "SU",…}
        34: {marketManager: "", phone: "07129-928630", advertisingCounty: null, regionShort: "SW",…}
        35: {marketManager: "", phone: "036041-309889", advertisingCounty: null, regionShort: "OS",…}
        36: {marketManager: "", phone: "0621-46082209", advertisingCounty: null, regionShort: "SW",…}
        37: {marketManager: "", phone: "089-99300731", advertisingCounty: null, regionShort: "SU",…}
        38: {marketManager: "Niklas Gerlach", phone: "0421/84919860", advertisingCounty: null, regionShort: "NO",…}
            address: {street: "Bahnhofsplatz", houseNumber: "42", postalCode: "28195", city: "Bremen, Stadt",…}
            advertisingCounty: null
            closedFrom: null
            closedUntil: null
            company: {name: "REWE Niklas Gerlach oHG", street: "Bahnhofsplatz 42", zipCode: "28195", city: "Bremen"}
            companyName: "REWE Niklas Gerlach oHG"
            dortmund: false
            externalLink: null
            firstOpeningDate: "2019-05-16"
            geoLocation: {latitude: 53.081348, longitude: 8.812994}
            holiday: null
            holidayDate: null
            id: 541755
            marketManager: "Niklas Gerlach"
            nahkauf: false
            openOnSunday: false
            opened: true
            openedUntil: {hour: 23, minute: 59, second: 0, nano: 0}
            openingHours: {condensed: [{days: "Mo-Fr", hours: "06:00 - 24:00"}, {days: "Sa", hours: "06:00 - 23:30"}],…}
                condensed: [{days: "Mo-Fr", hours: "06:00 - 24:00"}, {days: "Sa", hours: "06:00 - 23:30"}]
                    0: {days: "Mo-Fr", hours: "06:00 - 24:00"}
                        days: "Mo-Fr"
                        hours: "06:00 - 24:00"
                    1: {days: "Sa", hours: "06:00 - 23:30"}
                        days: "Sa"
                        hours: "06:00 - 23:30"
                dayAndTimeRanges: [{startDay: "MONDAY", endDay: "FRIDAY", startTime: {hour: 6, minute: 0, second: 0, nano: 0},…},…]
                    0: {startDay: "MONDAY", endDay: "FRIDAY", startTime: {hour: 6, minute: 0, second: 0, nano: 0},…}
                        closes: "23:59"
                        endDay: "FRIDAY"
                        endTime: {hour: 23, minute: 59, second: 0, nano: 0}
                        opens: "06:00"
                        seoRepresentation: "Mo-Fr 06:00-23:59"
                        startDay: "MONDAY"
                        startTime: {hour: 6, minute: 0, second: 0, nano: 0}
                    1: {startDay: "SATURDAY", endDay: null, startTime: {hour: 6, minute: 0, second: 0, nano: 0},…}
                        closes: "23:30"
                        endDay: null
                        endTime: {hour: 23, minute: 30, second: 0, nano: 0}
                        opens: "06:00"
                        seoRepresentation: "Sa 06:00-23:30"
                        startDay: "SATURDAY"
                        startTime: {hour: 6, minute: 0, second: 0, nano: 0}
            phone: "0421/84919860"
            pickupMarket: true
            regionShort: "NO"
            specialEnd: null
            specialOpening: null
            specialOpeningHours: null
            specialStart: null
            state: "Bremen"
            stateShort: "HB"
            temporaryClosed: false
            type: {name: "REWE", id: null}
            wawi: "41653792"
        39: {marketManager: "", phone: "02236-3945206", advertisingCounty: null, regionShort: "WE",…}
        40: {marketManager: "", phone: "0711-99718392", advertisingCounty: null, regionShort: "SW",…}
        41: {marketManager: "", phone: "07425-3275109", advertisingCounty: null, regionShort: "SW",…}
        42: {marketManager: "", phone: "05303/9797939", advertisingCounty: null, regionShort: "NO",…}
        43: {marketManager: "", phone: "069-13028727", advertisingCounty: null, regionShort: "MI",…}
        44: {marketManager: "", phone: "08231-340221", advertisingCounty: null, regionShort: "SU",…}
        45: {marketManager: "", phone: "06821-8691052", advertisingCounty: null, regionShort: "SW",…}
        46: {marketManager: "", phone: "0711-9973658", advertisingCounty: null, regionShort: "SW",…}
        47: {marketManager: "", phone: "07307-955001", advertisingCounty: null, regionShort: "SW",…}
        48: {marketManager: "", phone: "030-56298753", advertisingCounty: null, regionShort: "OS",…}
        49: {marketManager: "", phone: "0331-74001370", advertisingCounty: null, regionShort: "OS",…}
        50: {marketManager: "", phone: "089-91059059", advertisingCounty: null, regionShort: "SU",…}
        51: {marketManager: "", phone: "0941-46021047", advertisingCounty: null, regionShort: "SU",…}
        52: {marketManager: "", phone: "06486-900600", advertisingCounty: null, regionShort: "WE",…}
        53: {marketManager: "", phone: "06236-462809", advertisingCounty: null, regionShort: "SW",…}
        54: {marketManager: "", phone: "02065-892138", advertisingCounty: null, regionShort: "WE",…}
        55: {marketManager: "", phone: "07731-984576", advertisingCounty: null, regionShort: "SW",…}
        56: {marketManager: "David Hegemann", phone: "0211 213108", advertisingCounty: null, regionShort: "WE",…}
        57: {marketManager: "", phone: "030-31504210", advertisingCounty: null, regionShort: "OS",…}
        58: {marketManager: "", phone: "030-28390336", advertisingCounty: null, regionShort: "OS",…}
        59: {marketManager: "", phone: "06222-70351", advertisingCounty: null, regionShort: "SW",…}
        60: {marketManager: "", phone: "030-52279324", advertisingCounty: null, regionShort: "OS",…}
        61: {marketManager: "", phone: "07842-994114", advertisingCounty: null, regionShort: "SW",…}
        62: {marketManager: "", phone: "05162-900583", advertisingCounty: null, regionShort: "NO",…}
        63: {marketManager: "", phone: "0221-9320092", advertisingCounty: null, regionShort: "WE",…}
        64: {marketManager: "", phone: "07151-5027884", advertisingCounty: null, regionShort: "SW",…}
        65: {marketManager: "", phone: "07522-9155829", advertisingCounty: null, regionShort: "SW",…}
        66: {marketManager: "", phone: "07151-2051475", advertisingCounty: null, regionShort: "SW",…}
        67: {marketManager: "", phone: "08333-923938", advertisingCounty: null, regionShort: "SU",…}
        68: {marketManager: "", phone: "0511-5799402", advertisingCounty: null, regionShort: "NO",…}
        69: {marketManager: "", phone: "05481/997173", advertisingCounty: null, regionShort: "NO",…}
        70: {marketManager: "", phone: "07644-923407", advertisingCounty: null, regionShort: "SW",…}
        71: {marketManager: "", phone: "07305-927144", advertisingCounty: null, regionShort: "SW",…}
        72: {marketManager: "Vanessa Richter-Helms", phone: "04453-989620", advertisingCounty: null,…}
        73: {marketManager: "", phone: "0221-9541363", advertisingCounty: null, regionShort: "WE",…}
        74: {marketManager: "", phone: "02443-315490", advertisingCounty: null, regionShort: "WE",…}
        75: {marketManager: "", phone: "0201-8405411", advertisingCounty: null, regionShort: "WE",…}
        76: {marketManager: "", phone: "039954-2797030", advertisingCounty: null, regionShort: "OS",…}
        77: {marketManager: "", phone: "03381-211538", advertisingCounty: null, regionShort: "OS",…}
        78: {marketManager: "", phone: "07042-8171934", advertisingCounty: null, regionShort: "SW",…}
        79: {marketManager: "", phone: "02739/1547", advertisingCounty: null, regionShort: "WE",…}
        80: {marketManager: "", phone: "030-61709080", advertisingCounty: null, regionShort: "OS",…}
        81: {marketManager: "", phone: "0385-5557430", advertisingCounty: null, regionShort: "OS",…}
        82: {marketManager: "", phone: "0381-8651690", advertisingCounty: null, regionShort: "OS",…}
        83: {marketManager: "", phone: "0221-557492", advertisingCounty: null, regionShort: "WE",…}
        84: {marketManager: "Albion Karaxha", phone: "07545-1727", advertisingCounty: null, regionShort: "SW",…}
        85: {marketManager: "", phone: "03327-668970", advertisingCounty: null, regionShort: "OS",…}
        86: {marketManager: "", phone: "03831-66671130", advertisingCounty: null, regionShort: "OS",…}
        87: {marketManager: "", phone: "039601-3240130", advertisingCounty: null, regionShort: "OS",…}
        88: {marketManager: "", phone: "030-51062792", advertisingCounty: null, regionShort: "OS",…}
        89: {marketManager: "", phone: "06226-990814", advertisingCounty: null, regionShort: "SW",…}
        90: {marketManager: "", phone: "0331-2373520", advertisingCounty: null, regionShort: "OS",…}
        91: {marketManager: "", phone: "0395-430640", advertisingCounty: null, regionShort: "OS",…}
        92: {marketManager: "", phone: "039753-32330", advertisingCounty: null, regionShort: "OS",…}
        93: {marketManager: "", phone: "089-66002219", advertisingCounty: null, regionShort: "SU",…}
        94: {marketManager: "", phone: "0385-39576330", advertisingCounty: null, regionShort: "OS",…}
        95: {marketManager: "", phone: "030-78990490", advertisingCounty: null, regionShort: "OS",…}
        96: {marketManager: "", phone: "030-42016551", advertisingCounty: null, regionShort: "OS",…}
        97: {marketManager: "", phone: "030-54712219", advertisingCounty: null, regionShort: "OS",…}
        98: {marketManager: "", phone: "030-83202307", advertisingCounty: null, regionShort: "OS",…}
        99: {marketManager: "", phone: "0531-5160190", advertisingCounty: null, regionShort: "NO",…}
    [100 … 199]
    [200 … 299]
    [300 … 399]
    [400 … 498]
nextPage: 4
prevPage: 2
total: 3236

SyntaxError: invalid syntax (<ipython-input-1-23983c798c4c>, line 1)

## Code for setting up a database filled with all of REWE's opening hours, formatted to our liking:

In [ ]:
def set_up_rewe_database():
    API_URL = "https://www.rewe.de/market/content/marketsearch"
    headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36",
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                "accept-language": "en-US,en;q=0.9",
                "sec-fetch-dest": "document",
                "sec-fetch-mode": "navigate",
                "sec-fetch-site": "none",
                "sec-fetch-user": "?1",
                "upgrade-insecure-requests": "1" 
              }
    pageStoreCount = 1
    page = 0
    reweArray = []
    intermediaryDayKeys = { 'MONDAY' : 0, 'TUESDAY' : 1, 'WEDNESDAY' : 2, 'THURSDAY' : 3, 'FRIDAY' : 4, 'SATURDAY' : 5, 'SUNDAY' : 6, None : None}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    while pageStoreCount > 0:
        params = {  'searchString' : "REWE",
                    'pageSize' : 500,
                    'page' : page}
        try:
            req_url = API_URL + '?' + '&'.join(k + '=' + str(v) for k, v in params.items())
            rq = urllib.request.urlopen(urllib.request.Request(url=req_url, data=None, headers=headers))
            if rq.status != 200:
                return False
            data = rq.read().decode('utf-8')
            res = json.loads(data)
            if res["total"] == 0:
                return False
            res = res["markets"]
            pageStoreCount = len(res)
        except:
            return False
        # print(res[347]["openingHours"]["dayAndTimeRanges"])
        for index in range(len(res)):
            dayDoneSet = set()
            daysHeap = []
            openingHours = res[index]["openingHours"]["dayAndTimeRanges"]
            for dayRange in range(len(openingHours)):
                startDay = intermediaryDayKeys[openingHours[dayRange]["startDay"]]
                endDay = intermediaryDayKeys[openingHours[dayRange]["endDay"]]
                opens = openingHours[dayRange]["opens"]
                closes = openingHours[dayRange]["closes"]
                actualHours = [{'open' : opens, 'close' : closes}]
                #deal with 1 day ranges
                if endDay == None:
                    endDay = startDay
                #deal with wrap around windows e.g. sunday to tuesday
                if startDay > endDay:
                    startDay += -7
                for day in range(startDay, endDay+1):
                    if day < 0:
                        day += 7
                    #if statement to deal with incorrect responses where multiple ranges cover the same day (implemnted due to a bug in their system)
                    if day not in dayDoneSet:
                        dayDoneSet.add(day)
                        keyHours = {'day' : dayKeys[day],
                        'open' : True,
                        'hours' : actualHours
                        }
                        heapq.heappush(daysHeap, [day,keyHours])
            # simple heap method to check for any missing days (method which could be deployed to all other API functions quite easily)
            checkedUpToDay = -1
            for day in heapq.nsmallest(7, daysHeap):
                checkedUpToDay += 1
                while day[0] > checkedUpToDay:
                    closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                            'open' : False  }
                    heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
                    checkedUpToDay += 1
            while len(daysHeap) < 7:
                if len(daysHeap) > 0:
                    checkedUpToDay = daysHeap[-1][0] + 1
                else:
                    # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
                    checkedUpToDay = 0
                closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                            'open' : False  }
                heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            if len(daysHeap) != 7:
                return False
            hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
            coords = res[index]["geoLocation"]
            reweArray.append([(float(coords["latitude"]),float(coords["longitude"])), hoursArray])
        page += 1
    reweDB = pd.DataFrame(reweArray, columns=['Coordinates', 'OpeningHours'])
    reweDB.to_csv("REWE.csv")
    return True

## Code for retrieving REWE's opening hours data from our database (via Haversine):

In [ ]:
def get_rewe_data(lat, lng):
    desiredCoords = [lat, lng]
    reweDB = pd.read_csv("REWE.csv", index_col="Unnamed: 0", converters={'Coordinates': eval, 'OpeningHours' : eval})
    reweDB["Distances"] = haversine_vector([desiredCoords]*len(reweDB["Coordinates"]), list(reweDB["Coordinates"]))
    closestStoreIndex = reweDB["Distances"].idxmin()
    return reweDB["OpeningHours"][closestStoreIndex]

# Netto's API
**Note:** The function for this system also accepts lat and lng as length 1 arrays (simulated functioning overriding). In which case the storeDistance to the returned store is also returned ( in the form <code>[storeDisance, hoursArray]</code>), to allow the multi store get_netto_brands_data(lat, lng) function to call the different Netto functions and compare the distances of the stores returned.

**Note:** Not to be confused with Netto Marken-Discount, which is an entirely different brand with 4100 stores vs 342.

**Note:** This API returns **all** stores in Germany **only** upon calling. (For example does not return the 500+ stores located in Denmark)

**Note:** To retrieve the opening hours of a Netto store's for a given pair of coordinates and you are unsure of which of the two Companies own it, it is recommand to use the get_netto_brands_data(lat, lng) function instead, as this will return the hours of whichever store is closest from the union of the two sets of branches. (for example if using Fuzzywuzzy string matches you could match on Netto rather than distinguish the two confusingly named store brands)


## Base URL:
https://netto.de/umbraco/api/StoresData/StoresV2


## Sample API Call:
https://netto.de/umbraco/api/StoresData/StoresV2
## Sample Response:

In [ ]:
[0 … 99]
    0: {,…}
        address: {city: "Demmin", street: "Jarmener Strasse 57-58", zip: "17109", country: "DE", extra: null}
            city: "Demmin"
            country: "DE"
            extra: null
            street: "Jarmener Strasse 57-58"
            zip: "17109"
        attributes: {}
        coordinates: [13.0500393, 53.90632]
            0: 13.0500393
            1: 53.90632
        hours: [,…]
            0: {close: "2020-07-22T20:00:00", open: "2020-07-22T07:00:00", closed: false, date: "2020-07-22T00:00:00",…}
                close: "2020-07-22T20:00:00"
                closed: false
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0.19, 0.28, 0.38, 0.5, 0.39, 0.29, 0.28, 0.33, 0.33, 0.43, 0.4, 0.31, 0.26, 0, 0,…]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0.19
                    8: 0.28
                    9: 0.38
                    10: 0.5
                    11: 0.39
                    12: 0.29
                    13: 0.28
                    14: 0.33
                    15: 0.33
                    16: 0.43
                    17: 0.4
                    18: 0.31
                    19: 0.26
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-22T00:00:00"
                open: "2020-07-22T07:00:00"
                type: "store"
            1: {close: "2020-07-23T20:00:00", open: "2020-07-23T07:00:00", closed: false, date: "2020-07-23T00:00:00",…}
                close: "2020-07-23T20:00:00"
                closed: false
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0.19, 0.3, 0.54, 0.58, 0.44, 0.36, 0.2, 0.28, 0.31, 0.43, 0.43, 0.34, 0.23, 0, 0,…]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0.19
                    8: 0.3
                    9: 0.54
                    10: 0.58
                    11: 0.44
                    12: 0.36
                    13: 0.2
                    14: 0.28
                    15: 0.31
                    16: 0.43
                    17: 0.43
                    18: 0.34
                    19: 0.23
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-23T00:00:00"
                open: "2020-07-23T07:00:00"
                type: "store"
            2: {close: "2020-07-24T20:00:00", open: "2020-07-24T07:00:00", closed: false, date: "2020-07-24T00:00:00",…}
                close: "2020-07-24T20:00:00"
                closed: false
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0.18, 0.29, 0.51, 0.57, 0.54, 0.34, 0.33, 0.29, 0.45, 0.43, 0.43, 0.35, 0.28, 0,…]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0.18
                    8: 0.29
                    9: 0.51
                    10: 0.57
                    11: 0.54
                    12: 0.34
                    13: 0.33
                    14: 0.29
                    15: 0.45
                    16: 0.43
                    17: 0.43
                    18: 0.35
                    19: 0.28
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-24T00:00:00"
                open: "2020-07-24T07:00:00"
                type: "store"
            3: {close: "2020-07-25T20:00:00", open: "2020-07-25T07:00:00", closed: false, date: "2020-07-25T00:00:00",…}
                close: "2020-07-25T20:00:00"
                closed: false
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0.2, 0.25, 0.44, 0.55, 0.39, 0.32, 0.31, 0.35, 0.31, 0.36, 0.38, 0.4, 0.31, 0, 0,…]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0.2
                    8: 0.25
                    9: 0.44
                    10: 0.55
                    11: 0.39
                    12: 0.32
                    13: 0.31
                    14: 0.35
                    15: 0.31
                    16: 0.36
                    17: 0.38
                    18: 0.4
                    19: 0.31
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-25T00:00:00"
                open: "2020-07-25T07:00:00"
                type: "store"
            4: {close: null, open: null, closed: true, date: "2020-07-26T00:00:00", type: "store",…}
                close: null
                closed: true
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0
                    8: 0
                    9: 0
                    10: 0
                    11: 0
                    12: 0
                    13: 0
                    14: 0
                    15: 0
                    16: 0
                    17: 0
                    18: 0
                    19: 0
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-26T00:00:00"
                open: null
                type: "store"
            5: {close: "2020-07-27T20:00:00", open: "2020-07-27T07:00:00", closed: false, date: "2020-07-27T00:00:00",…}
                close: "2020-07-27T20:00:00"
                closed: false
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0.31, 0.35, 0.51, 0.54, 0.42, 0.36, 0.33, 0.37, 0.49, 0.52, 0.48, 0.35, 0.23, 0,…]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0.31
                    8: 0.35
                    9: 0.51
                    10: 0.54
                    11: 0.42
                    12: 0.36
                    13: 0.33
                    14: 0.37
                    15: 0.49
                    16: 0.52
                    17: 0.48
                    18: 0.35
                    19: 0.23
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-27T00:00:00"
                open: "2020-07-27T07:00:00"
                type: "store"
            6: {close: "2020-07-28T20:00:00", open: "2020-07-28T07:00:00", closed: false, date: "2020-07-28T00:00:00",…}
                close: "2020-07-28T20:00:00"
                closed: false
                customerFlow: [0, 0, 0, 0, 0, 0, 0, 0.17, 0.32, 0.48, 0.54, 0.4, 0.36, 0.31, 0.41, 0.36, 0.44, 0.44, 0.36, 0.18, 0,…]
                    0: 0
                    1: 0
                    2: 0
                    3: 0
                    4: 0
                    5: 0
                    6: 0
                    7: 0.17
                    8: 0.32
                    9: 0.48
                    10: 0.54
                    11: 0.4
                    12: 0.36
                    13: 0.31
                    14: 0.41
                    15: 0.36
                    16: 0.44
                    17: 0.44
                    18: 0.36
                    19: 0.18
                    20: 0
                    21: 0
                    22: 0
                    23: 0
                date: "2020-07-28T00:00:00"
                open: "2020-07-28T07:00:00"
                type: "store"
        id: "c9d0231d-d88f-4aa8-9e62-2272342d3e63"
        name: "Netto Demmin"
        type: "Point"
        url: "/geschaefte/demmin/jarmener-strasse-57-58/c9d0231dd88f4aa89e622272342d3e63"
    1: {address: {city: "Malchin", street: "Teichstrasse 8", zip: "17139", country: "DE", extra: null},…}
    2: {,…}
    3: {address: {city: "Dargun", street: "Schloßstraße 73", zip: "17159", country: "DE", extra: null},…}
    4: {address: {city: "Berlin", street: "Pistoriusstrasse 13", zip: "13086", country: "DE", extra: null},…}
    5: {,…}
    6: {,…}
    7: {,…}
    8: {,…}
    9: {address: {city: "Koserow", street: "Hauptstraße 87", zip: "17459", country: "DE", extra: null},…}
    10: {,…}
    11: {,…}
    12: {,…}
    13: {address: {city: "Mahlow", street: "Ibsenstrasse 69", zip: "15831", country: "DE", extra: null},…}
    14: {address: {city: "Friedland", street: "Riemannstrasse 34", zip: "17098", country: "DE", extra: null},…}
    15: {,…}
    16: {address: {city: "Jarmen", street: "Müssentiner Weg 1", zip: "17126", country: "DE", extra: null},…}
    17: {,…}
    18: {,…}
    19: {,…}
    20: {address: {city: "Anklam", street: "Lübecker Straße 19", zip: "17389", country: "DE", extra: null},…}
    21: {,…}
    22: {,…}
    23: {address: {city: "Penzlin", street: "Bahnhofplatz 5", zip: "17217", country: "DE", extra: null},…}
    24: {address: {city: "Lübbenau", street: "Bahnhofstraße 21", zip: "03222", country: "DE", extra: null},…}
    25: {address: {city: "Beeskow", street: "Luchstraße 32", zip: "15848", country: "DE", extra: null},…}
    26: {,…}
    27: {,…}
    28: {,…}
    29: {address: {city: "Berlin", street: "Kiefholzstraße 420", zip: "12435", country: "DE", extra: null},…}
    30: {,…}
    31: {address: {city: "Parchim", street: "Neuhofer Weiche 1", zip: "19370", country: "DE", extra: null},…}
    32: {address: {city: "Strasburg", street: "Lindenstrasse 5", zip: "17335", country: "DE", extra: null},…}
    33: {address: {city: "Parchim", street: "Westring 39", zip: "19370", country: "DE", extra: null},…}
    34: {,…}
    35: {address: {city: "Berlin", street: "Glienicker Weg 126", zip: "12489", country: "DE", extra: null},…}
    36: {,…}
    37: {address: {city: "Brüel", street: "Am Mühlenberg 1a", zip: "19412", country: "DE", extra: null},…}
    38: {,…}
    39: {address: {city: "Barth", street: "Hölzern-Kreuz-Weg 13b", zip: "18356", country: "DE", extra: null},…}
    40: {,…}
    41: {address: {city: "Rostock", street: "Zum Laakkanal 3", zip: "18109", country: "DE", extra: null},…}
    42: {,…}
    43: {address: {city: "Rostock", street: "Brahestraße 44", zip: "18059", country: "DE", extra: null},…}
    44: {address: {city: "Rostock", street: "Ziolkowskistraße 1", zip: "18059", country: "DE", extra: null},…}
    45: {,…}
    46: {,…}
    47: {,…}
    48: {address: {city: "Cottbus", street: "Leuthener Straße 7", zip: "03050", country: "DE", extra: null},…}
    49: {address: {city: "Velten", street: "Lindenstrasse 8", zip: "16727", country: "DE", extra: null},…}
    50: {,…}
    51: {,…}
    52: {address: {city: "Eggersdorf", street: "Am Fuchsbau 3", zip: "15345", country: "DE", extra: null},…}
    53: {address: {city: "Binz", street: "Proraer Chaussee 2", zip: "18609", country: "DE", extra: null},…}
    54: {,…}
    55: {,…}
    56: {,…}
    57: {address: {city: "Templin", street: "Lychener Straße 23a", zip: "17268", country: "DE", extra: null},…}
    58: {,…}
    59: {address: {city: "Wittenberge", street: "Turmstraße 13", zip: "19322", country: "DE", extra: null},…}
    60: {address: {city: "Neuenhagen", street: "Lindenstrasse 61", zip: "15366", country: "DE", extra: null},…}
    61: {,…}
    62: {address: {city: "Berlin", street: "Alt Heiligensee 8", zip: "13503", country: "DE", extra: null},…}
    63: {address: {city: "Berlin", street: "Am Gehrensee 2", zip: "13057", country: "DE", extra: null},…}
    64: {,…}
    65: {address: {city: "Berlin", street: "Ribnitzer Strasse 24", zip: "13051", country: "DE", extra: null},…}
    66: {address: {city: "Berlin", street: "Eschengraben 47", zip: "13189", country: "DE", extra: null},…}
    67: {,…}
    68: {,…}
    69: {,…}
    70: {,…}
    71: {,…}
    72: {,…}
    73: {address: {city: "Wittstock", street: "Polthierstraße 14", zip: "16909", country: "DE", extra: null},…}
    74: {address: {city: "Salzwedel", street: "Bahnhofstraße 2", zip: "29410", country: "DE", extra: null},…}
    75: {,…}
    76: {,…}
    77: {,…}
    78: {address: {city: "Hagenow", street: "Goethestrasse 6", zip: "19230", country: "DE", extra: null},…}
    79: {,…}
    80: {,…}
    81: {,…}
    82: {,…}
    83: {address: {city: "Pasewalk", street: "Bahnhofstraße 17", zip: "17309", country: "DE", extra: null},…}
    84: {,…}
    85: {,…}
    86: {,…}
    87: {address: {city: "Burg", street: "Friedensstrasse 49", zip: "39288", country: "DE", extra: null},…}
    88: {,…}
    89: {,…}
    90: {,…}
    91: {,…}
    92: {,…}
    93: {address: {city: "Nauen", street: "Dammstraße 7a", zip: "14641", country: "DE", extra: null},…}
    94: {,…}
    95: {,…}
    96: {address: {city: "Löcknitz", street: "Chaussee Straße 37", zip: "17321", country: "DE", extra: null},…}
    97: {,…}
    98: {,…}
    99: {address: {city: "Stendal", street: "Stadtseeallee 31", zip: "39576", country: "DE", extra: null},…}
    [100 … 199]
    [200 … 299]
    [300 … 341]

# Code for setting up a database filled with all of Netto's opening hours, formatted to our liking:

In [ ]:
def set_up_netto_database():
    API_URL = "https://netto.de/umbraco/api/StoresData/StoresV2"
    nettoArray = []
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    try:
        rq = requests.get("https://netto.de/umbraco/api/StoresData/StoresV2")
        res = rq.json()
        if len(res) == 0:
            return False
    except:
        return False
    for index in range(len(res)):
        dayDoneSet = set()
        daysHeap = []
        openingHours = res[index]["hours"]
        for dayIndex in range(len(openingHours)):
            #potential weakness if a bugged date is retrieved and the datetime parser raises an exception?
            day = parser.parse(openingHours[dayIndex]["date"]).weekday()
            if day not in dayDoneSet:
                try:
                    dayDoneSet.add(day)
                    #keyHours processing code
                    if openingHours[dayIndex]["closed"] == False:
                        opens = parser.parse(openingHours[dayIndex]["open"]).strftime("%H:%M")
                        closes = parser.parse(openingHours[dayIndex]["close"]).strftime("%H:%M")
                        actualHours = {'open' : opens,
                                'close' : closes}
                        keyHours = {'day' : dayKeys[day],
                        'open' : True,
                        'hours' : [actualHours]
                        }
                        heapq.heappush(daysHeap, [day,keyHours])
                    else:
                        closedDayHours = {  'day' : dayKeys[day],
                        'open' : False  }
                        heapq.heappush(daysHeap, [day, closedDayHours])
                except:
                    closedDayHours = {  'day' : dayKeys[day],
                        'open' : False  }
                    heapq.heappush(daysHeap, [day, closedDayHours])
        #Check for any missing days and ensure all days are in order for insertion into our database
        checkedUpToDay = -1
        for day in heapq.nsmallest(7, daysHeap):
            checkedUpToDay += 1
            while day[0] > checkedUpToDay:
                closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                        'open' : False  }
                heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
                checkedUpToDay += 1
        while len(daysHeap) < 7:
            if len(daysHeap) > 0:
                checkedUpToDay = daysHeap[-1][0] + 1
            else:
                # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
                checkedUpToDay = 0
                closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                            'open' : False  }
                heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
        if len(daysHeap) != 7:
            return False
        hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
        coords = res[index]["coordinates"]
        nettoArray.append([(float(coords[1]),float(coords[0])), hoursArray])
    nettoDB = pd.DataFrame(nettoArray, columns=['Coordinates', 'OpeningHours'])
    nettoDB.to_csv("Netto.csv")
    return True

# Code for retrieving Netto's opening hours data from our database (via Haversine):

In [ ]:
def get_netto_data(lat, lng):
    #pass lat and lng as arrays of length 1 if you want the storeDistance to be returned too
    if isinstance(lat, list) :
        returnDistanceToo = True
        lat = lat[0]
        lng = lng[0]
    else:
        returnDistanceToo = False
    desiredCoords = [lat, lng]
    nettoDB = pd.read_csv("Netto.csv", index_col="Unnamed: 0", converters={'Coordinates': eval, 'OpeningHours' : eval})
    nettoDB["Distances"] = haversine_vector([desiredCoords]*len(nettoDB["Coordinates"]), list(nettoDB["Coordinates"]))
    closestStoreIndex = nettoDB["Distances"].idxmin()
    if returnDistanceToo:
        return nettoDB["Distances"][closestStoreIndex], nettoDB["OpeningHours"][closestStoreIndex]
    else:
        return nettoDB["OpeningHours"][closestStoreIndex]

# Netto Marken-Discount's API (Germany)
**Note:** The function for this system also accepts lat and lng as length 1 arrays (simulated functioning overriding). In which case the storeDistance to the returned store is also returned ( in the form <code>[storeDisance, hoursArray]</code>), to allow the multi store get_netto_brands_data(lat, lng) function to call the different Netto functions and compare the distances of the stores returned.


**Note:** To retrieve the opening hours of a Netto store's for a given pair of coordinates and you are unsure of which of the two Companies own it, it is recommand to use the get_netto_brands_data(lat, lng) function instead, as this will return the hours of whichever store is closest from the union of the two sets of branches. (for example if using Fuzzywuzzy string matches you could match on Netto rather than distinguish the two confusingly named store brands)
## Base URL:
https://www.netto-online.de/INTERSHOP/web/WFS/Plus-NettoDE-Site/de_DE/-/EUR/ViewNettoStoreFinder-GetStoreItems
## Parameters:
These four parameters form the boundary (South, North, West, East) of the map viewport within which to return stores(website submits these via Form Data):
* s = lower latitude boundary
* n = upper latitude boundary
* w = lower longitude boundary
* e = upper longitude boundary

**Note:** There is a maximum size allowed on the website.

## Sample API Call:
https://www.netto-online.de/INTERSHOP/web/WFS/Plus-NettoDE-Site/de_DE/-/EUR/ViewNettoStoreFinder-GetStoreItems?w=13.3&e=13.5&s=52.4&n=52.6
## Sample Response:

In [ ]:
[{store_id: "5544", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…},…]
    0: {store_id: "5544", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
        alternative_stores_id: []
        city: "Berlin-Treptow"
        coord_latitude: "52.479225"
        coord_longitude: "13.481276"
        has_bs: true
        is_city: false
        is_closed: false
        is_delivery: "0"
        is_pickup: false
        is_user_store: false
        meat_market_type: "SELF-SERVICE"
        post_code: "12435"
        region_id: "37"
        state: "Berlin"
        store_extra_info_long: ""
        store_id: "5544"
        store_name: "Netto Filiale"
        store_opening: "Mo.-Sa.: 7.00 - 22.00 Uhr<br />So.: geschlossen<br />"
        store_type_id: "1"
        street: "Dammweg 56"
    1: {store_id: "5341", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    2: {store_id: "5358", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    3: {store_id: "5008", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    4: {store_id: "5026", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    5: {store_id: "5032", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    6: {store_id: "5040", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    7: {store_id: "5115", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    8: {store_id: "5142", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    9: {store_id: "5228", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    10: {store_id: "5250", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    11: {store_id: "7779", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    12: {store_id: "7783", is_user_store: false, region_id: "37", store_type_id: "1", is_city: true,…}
    13: {store_id: "7785", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    14: {store_id: "7788", is_user_store: false, region_id: "37", store_type_id: "1", is_city: true,…}
    15: {store_id: "7829", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    16: {store_id: "7428", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    17: {store_id: "7429", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    18: {store_id: "7446", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    19: {store_id: "7448", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    20: {store_id: "7455", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}
    21: {store_id: "7456", is_user_store: false, region_id: "37", store_type_id: "1", is_city: true,…}
    22: {store_id: "7457", is_user_store: false, region_id: "37", store_type_id: "1", is_city: false,…}

# Code for retrieving Netto Marken-Discount's opening hours data:

In [ ]:
def get_netto_marken_discount_data(lat, lng):
    #pass lat and lng as arrays of length 1 if you want the storeDistance to be returned too
    if isinstance(lat, list) :
        returnDistanceToo = True
        lat = lat[0]
        lng = lng[0]
    else:
        returnDistanceToo = False
    API_URL = "https://www.netto-online.de/INTERSHOP/web/WFS/Plus-NettoDE-Site/de_DE/-/EUR/ViewNettoStoreFinder-GetStoreItems"
    #The conversion rates below have been estimated through experimentation, and should approximately preserve radius (although the geometry will be a cross between a square and a circle)
    params = {  's' : float(lat) - radius / 70,
                'n' : float(lat) + radius / 70,
                'w' : float(lng) - radius / 110,
                'e' : float(lng) + radius / 110
             }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        return False
    try:
        res = rq.json()
        if len(res) == 0:
            print(2)
            return False
    except:
        return False
    storeHeap = []
    for index in range(len(res)):
        storeLat, storeLng = float(res[index]["coord_latitude"]), float(res[index]["coord_longitude"])
        #compute distance between the two points using the haversine function
        storeDistance = haversine((lat, lng),(storeLat, storeLng))
        heapq.heappush(storeHeap, [storeDistance,res[index]])
    res = heapq.nsmallest(limit, storeHeap)
    i = 0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!!
    #e.g.:
    # while True:
    #     if name != res[0]['name'] or res[0]['other_name']:
    #         i += 1
    #     else:
    #         break
    intermediaryDayKeys = { 'Mo.' : 0, 'Di.' : 1, 'Mi.' : 2, 'Do.' : 3, 'Fr.' : 4, 'Sa.' : 5, 'So.' : 6, None : None}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    dayDoneSet = set()
    daysHeap = []
    storeDistance = res[i][0]
    openingHours = res[i][1]["store_opening"]
    dayRanges = openingHours.split("<br />")[:-1]
    for dayRange in range(len(dayRanges)):
        daysAndTimes = dayRanges[dayRange].split(":")
        days = daysAndTimes[0]
        days = days.split("-")
        startDay = intermediaryDayKeys[days[0]]
        if len(days) == 1:
            endDay = startDay
        elif len(days) == 2:
            endDay = intermediaryDayKeys[days[1]]
        else:
            return False
        #deal with wrap around windows e.g. sunday to tuesday
        if startDay > endDay:
            startDay += -7
        times = daysAndTimes[1]
        if "geschlossen" in times:
            for day in range(startDay, endDay+1):
                if day < 0:
                    day += 7
                #if statement to deal with incorrect responses where multiple ranges cover the same day (implemnted due to a bug in their system)
                if day not in dayDoneSet:
                    dayDoneSet.add(day)
                    closedDayHours = {  'day' : dayKeys[day],
                                'open' : False  }
                    heapq.heappush(daysHeap, [day, closedDayHours])
        else:
            try:
                times = times.replace(".",":")
                times = times.split()
                if len(times) != 4:
                    #means this method is non exhaustive (e.g. multiple time slots) and must be amended 
                    return False
                opens = times[0]
                closes = times[2]
                if len(opens) == 4:
                    opens = "0" + opens
                if len(closes) == 4:
                    closes = "0" + closes
                if len(opens) != 5 or len(closes) != 5:
                    #means this method is non exhaustive (e.g. doesnt catch store closure) and must be amended 
                    return False
                actualHours = [{'open' : opens, 'close' : closes}]
                for day in range(startDay, endDay+1):
                    if day < 0:
                        day += 7
                    #if statement to deal with incorrect responses where multiple ranges cover the same day (implemnted due to a bug in their system)
                    if day not in dayDoneSet:
                        dayDoneSet.add(day)
                        keyHours = {'day' : dayKeys[day],
                        'open' : True,
                        'hours' : actualHours
                        }
                        heapq.heappush(daysHeap, [day,keyHours])
            except:
                for day in range(startDay, endDay+1):
                    if day < 0:
                        day += 7
                    #if statement to deal with incorrect responses where multiple ranges cover the same day (implemnted due to a bug in their system)
                    if day not in dayDoneSet:
                        dayDoneSet.add(day)
                        closedDayHours = {  'day' : dayKeys[day],
                                    'open' : False  }
                        heapq.heappush(daysHeap, [day, closedDayHours])
    # simple heap method to check for any missing days (method which could be deployed to all other API functions quite easily)
    checkedUpToDay = -1
    for day in heapq.nsmallest(7, daysHeap):
        checkedUpToDay += 1
        while day[0] > checkedUpToDay:
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            checkedUpToDay += 1
    while len(daysHeap) < 7:
        if len(daysHeap) > 0:
            checkedUpToDay = daysHeap[-1][0] + 1
        else:
            # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
            checkedUpToDay = 0
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                        'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
    if len(daysHeap) != 7:
        return False
    hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
    if returnDistanceToo:
        return storeDistance, hoursArray
    else:
        return hoursArray

# Code for retrieving opening hours data of whichever store is closest out of the two Netto Brands:

In [ ]:
def get_netto_brands_data(lat, lng):
    netto = get_netto_data([lat], [lng])
    nettoMD = get_netto_marken_discount_data([lat], [lng])
    if netto[0] < nettoMD[0]:
        return netto[1]
    else:
        return nettoMD[1]

# EDEKA's API (Germany)
## Base URL:
https://www.edeka.de/api/marketsearch/markets
## Parameters:
* searchstring = address to search around (the parameter the website uses)          
**OR**
* coordinates = hidden parameter which can be used to supply coordinates in the following form "lat={Latitude}&lon={Longitude}"

## Sample API Call:
https://www.edeka.de/api/marketsearch/markets?coordinates=lat%3D52.628684799999995%26lon%3D1.3369343999999999
## Sample Response:

In [ ]:
{totalCount: 52, limit: 10, offset: 0,…}
limit: 10
markets: [{id: 8001594, name: "EDEKA Friedrichstraße", distributionChannel: {name: "EDEKA", type: "EDEKA"},…},…]
    0: {id: 8001594, name: "EDEKA Friedrichstraße", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
        businessHours: {monday: {weekday: "MONDAY", from: "07:00", to: "22:00"},…}
            friday: {weekday: "FRIDAY", from: "07:00", to: "22:00"}
                from: "07:00"
                to: "22:00"
                weekday: "FRIDAY"
            monday: {weekday: "MONDAY", from: "07:00", to: "22:00"}
                from: "07:00"
                to: "22:00"
                weekday: "MONDAY"
            saturday: {weekday: "SATURDAY", from: "07:00", to: "22:00"}
                from: "07:00"
                to: "22:00"
                weekday: "SATURDAY"
            thursday: {weekday: "THURSDAY", from: "07:00", to: "22:00"}
                from: "07:00"
                to: "22:00"
                weekday: "THURSDAY"
            tuesday: {weekday: "TUESDAY", from: "07:00", to: "22:00"}
                from: "07:00"
                to: "22:00"
                weekday: "TUESDAY"
            wednesday: {weekday: "WEDNESDAY", from: "07:00", to: "22:00"}
                from: "07:00"
                to: "22:00"
                weekday: "WEDNESDAY"
        contact: {,…}
            address: {city: {name: "Berlin", zipCode: "10969"}, street: "Friedrichstr. 246", federalState: "Berlin"}
                city: {name: "Berlin", zipCode: "10969"}
                    name: "Berlin"
                    zipCode: "10969"
                federalState: "Berlin"
                street: "Friedrichstr. 246"
            emailAddress: "edeka.friedrichstrasse@minden.edeka.de"
            facsimileNumber: "+493025295872"
            phoneNumber: "+493025295870"
        coordinates: {lat: "52.50003299999999", lon: "13.3913285"}
            lat: "52.50003299999999"
            lon: "13.3913285"
        distributionChannel: {name: "EDEKA", type: "EDEKA"}
            name: "EDEKA"
            type: "EDEKA"
        id: 8001594
        imprint: {companyName: "EDEKA Minden-Hannover Stiftung & Co. KG",…}
            address: {city: {name: "Minden", zipCode: "32427"}, street: "Wittelsbacherallee 61"}
                city: {name: "Minden", zipCode: "32427"}
                    name: "Minden"
                    zipCode: "32427"
                street: "Wittelsbacherallee 61"
            companyName: "EDEKA Minden-Hannover Stiftung & Co. KG"
        name: "EDEKA Friedrichstraße"
        nationalOffers: true
        services: [{id: "AUSBILDENDER_BETRIEB", name: "Ausbildender Betrieb", abbreviation: "a"},…]
            0: {id: "AUSBILDENDER_BETRIEB", name: "Ausbildender Betrieb", abbreviation: "a"}
                abbreviation: "a"
                id: "AUSBILDENDER_BETRIEB"
                name: "Ausbildender Betrieb"
            1: {id: "BACKSTATION", name: "Backstation", abbreviation: "bs"}
                abbreviation: "bs"
                id: "BACKSTATION"
                name: "Backstation"
            2: {id: "DEUTSCHLAND_CARD", name: "DeutschlandCard", abbreviation: "d"}
                abbreviation: "d"
                id: "DEUTSCHLAND_CARD"
                name: "DeutschlandCard"
            3: {id: "EDEKA_MOBIL", name: "EDEKA mobil", abbreviation: "em"}
                abbreviation: "em"
                id: "EDEKA_MOBIL"
                name: "EDEKA mobil"
            4: {id: "ERNAEHRUNGSSERVICE", name: "Ernährungsservice", abbreviation: "e"}
                abbreviation: "e"
                id: "ERNAEHRUNGSSERVICE"
                name: "Ernährungsservice"
            5: {id: "FLEISCH_WURSTTHEKE", name: "Fleisch-& Wursttheke", abbreviation: "fw"}
                abbreviation: "fw"
                id: "FLEISCH_WURSTTHEKE"
                name: "Fleisch-& Wursttheke"
            6: {id: "GLUTENFREI", name: "Glutenfrei", abbreviation: "gf"}
                abbreviation: "gf"
                id: "GLUTENFREI"
                name: "Glutenfrei"
            7: {id: "HANDYAUFLADUNG", name: "Handy-Aufladung", abbreviation: "ha"}
                abbreviation: "ha"
                id: "HANDYAUFLADUNG"
                name: "Handy-Aufladung"
            8: {id: "HAUSHALTSWAREN", name: "Haushaltswaren", abbreviation: "nf"}
                abbreviation: "nf"
                id: "HAUSHALTSWAREN"
                name: "Haushaltswaren"
            9: {id: "KREDITKARTE_AKZEPTIERT", name: "Kreditkarte akzeptiert", abbreviation: "ka"}
                abbreviation: "ka"
                id: "KREDITKARTE_AKZEPTIERT"
                name: "Kreditkarte akzeptiert"
            10: {id: "KAESETHEKE", name: "Käsetheke", abbreviation: "k"}
                abbreviation: "k"
                id: "KAESETHEKE"
                name: "Käsetheke"
            11: {id: "LAKTOSEFREI", name: "Laktosefrei", abbreviation: "lf"}
                abbreviation: "lf"
                id: "LAKTOSEFREI"
                name: "Laktosefrei"
            12: {id: "MOBILE_HANDY_COUPONS", name: "Mobile Handy-Coupons", abbreviation: "mhc"}
                abbreviation: "mhc"
                id: "MOBILE_HANDY_COUPONS"
                name: "Mobile Handy-Coupons"
            13: {id: "MOBILES_BEZAHLEN", name: "Mobiles Bezahlen per Handy", abbreviation: "mb"}
                abbreviation: "mb"
                id: "MOBILES_BEZAHLEN"
                name: "Mobiles Bezahlen per Handy"
            14: {id: "VEGAN", name: "Vegan", abbreviation: "vega"}
                abbreviation: "vega"
                id: "VEGAN"
                name: "Vegan"
            15: {id: "VEGETARISCH", name: "Vegetarisch", abbreviation: "vege"}
                abbreviation: "vege"
                id: "VEGETARISCH"
                name: "Vegetarisch"
        url: "https://www.edeka.de/eh/minden-hannover/edeka-friedrichstraße-friedrichstr.-246/index.jsp"
        validFrom: "09.06.2017"
    1: {id: 8001540, name: "EDEKA Ritterstraße", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    2: {id: 10001148, name: "EDEKA Stresemannstr.", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    3: {id: 128443, name: "EDEKA Ungefroren", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    4: {id: 10001525, name: "EDEKA Berlin Yorckstraße", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    5: {id: 8001596, name: "EDEKA Leipziger Platz", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    6: {id: 8001532, name: "EDEKA Annenstraße", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    7: {id: 8001539, name: "EDEKA Bergmannstraße", distributionChannel: {name: "EDEKA", type: "EDEKA"},…}
    8: {id: 20666, name: "EDEKA Siebert", distributionChannel: {name: "EDEKA", type: "EDEKA"}, contact: {,…},…}
    9: {id: 8000841, name: "EDEKA Brehm", distributionChannel: {name: "EDEKA", type: "EDEKA"}, contact: {,…},…}
offset: 0
totalCount: 52
_links: {self: {rel: "self",…}, next: {rel: "next",…}}
next: {rel: "next",…}
href: "/api/marketsearch/markets?coordinates=lat%3D52.503705%26lon%3D13.395646&page=1&size=10"
rel: "next"
self: {rel: "self",…}
href: "/api/marketsearch/markets?coordinates=lat%3D52.503705%26lon%3D13.395646&page=0&size=10"
rel: "self"

# Code for retrieving EDEKA's opening hours data:

In [ ]:
def get_edeka_data(lat, lng):
    API_URL = "https://www.edeka.de/api/marketsearch/markets"
    params = {  'coordinates' : "lat={0}&lon={1}".format(lat, lng)}
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()
        if res["totalCount"] == 0 or len(res['markets']) == 0:
            return False
        res = res["markets"]
    except:
        return False
    i = 0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0]["name"])
    #e.g.:
    # while True:
    #     if name != res[i]["name"]:
    #         i += 1
    #     else:
    #         break 
    storeLat, storeLng = float(res[i]["coordinates"]["lat"]), float(res[i]["coordinates"]["lon"])
    #compute distance between the two points using the haversine function
    storeDistance = haversine((lat, lng),(storeLat, storeLng))
    if storeDistance > radius:
        return False
    openingHours = res[i]["businessHours"]
    dayKeys = { 'monday' : 'Monday', 'tuesday' : 'Tuesday', 'wednesday' : 'Wednesday', 'thursday' : 'Thursday', 'friday' : 'Friday', 'saturday' : 'Saturday', 'sunday' : 'Sunday'}
    hoursArray = []
    for key in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
        try:
            actualHours = { 'open' : openingHours[key]['from'],
                            'close' : openingHours[key]['to']}
            keyHours = {'day' : dayKeys[key],
                    'open' : True,
                    'hours' : [actualHours]
                    }
        except:
            keyHours = {'day' : dayKeys[key],
                    'open' : False}
        hoursArray.append(keyHours)
    return hoursArray


# Sainsbury's API:
## Documentation:
### Base URL:
https://stores.sainsburys.co.uk/api/v1/stores/
### Parameters:
* fields: Not sure what that means. Default value: <code>fields=slfe-list-2.21</code>
* api_client_id: Not sure either. Default value: <code>api_client_id=slfe</code>
* lat: Latitude of search area. Example: <code>lat=51.5073509</code>
* lon: Longitude of search area. Example: <code>lon=-0.1277583</code>
* limit: Stores displayed per page. Default value: <code>limit=25</code>
* store_type: Type of stores displayed. Can contain *main* for superstores and/or *local* for local stores. Example: <code>stores_type=main,local</code>
* sort: How results are sorted. Only found *by_distance* so far. Default value: <code>sort=by_distance</code>
* within: Max distance for which results should be shown, Measured in Miles (**tested**). Seems to take a value between 0 and 500. Example: <code>within=15</code>
* page: Page of paginated results. Example: <code>page=1</code>

### Sample API Call:
https://stores.sainsburys.co.uk/api/v1/stores/?fields=slfe-list-2.21&api_client_id=slfe&lat=52.6476395&lon=1.2932064&limit=25&store_type=main%2Clocal&sort=by_distance&within=15&page=1

### Sample Response:

In [ ]:
page_meta: {limit: 25, offset: 0, total: 6}
results: [{additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…},…]
    0: {additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…}
        additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null}
            main_store_branded_as_local: null
            restaurant_branded_as_fresh_kitchen: null
        closed: null
        code: "4359"
        contact: {address1: "9 - 13 Stephens Street", address2: "", city: "Norwich", country: "England",…}
            address1: "9 - 13 Stephens Street"
            address2: ""
            city: "Norwich"
            country: "England"
            country_code: "GB"
            country_name: "United Kingdom"
            county: ""
            fax_number: ""
            manager: "Vicky Hampson"
            post_code: "NR1 3QN"
            state: ""
            telephone: "01603764089"
        distance: 1.5303873646684123
        district_code: "4359"
        exception_times: []
        location: {lat: "52.62549", lon: "1.29324"}
            lat: "52.62549"
            lon: "1.29324"
        name: "Nwch St Stpns St Loc"
        open: {current_date: "2020-07-18", current_time: "13:10", is_open: true, open_until: "23:00",…}
            current_date: "2020-07-18"
            current_time: "13:10"
            is_open: true
            open_until: "23:00"
            today: {24hrs: false, end_time: "23:00", exception: false, start_time: "07:00",…}
                24hrs: false
                end_time: "23:00"
                exception: false
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            tomorrow: {24hrs: false, end_time: "23:00", exception: false, start_time: "07:00",…}
                24hrs: false
                end_time: "23:00"
                exception: false
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
        opening_times: [{day: 0, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]},…]
            0: {day: 0, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 0
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            1: {day: 1, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 1
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            2: {day: 2, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 2
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            3: {day: 3, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 3
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            4: {day: 4, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 4
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            5: {day: 5, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 5
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
            6: {day: 6, end_time: "23:00", start_time: "07:00", times: [{end_time: "23:00", start_time: "07:00"}]}
                day: 6
                end_time: "23:00"
                start_time: "07:00"
                times: [{end_time: "23:00", start_time: "07:00"}]
                    0: {end_time: "23:00", start_time: "07:00"}
                        end_time: "23:00"
                        start_time: "07:00"
        opening_times_message: null
        originator_id: null
        other_name: "Norwich St Stephens Street Local"
        store_type: "local"
    1: {additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…}
    2: {additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…}
    3: {additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…}
    4: {additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…}
    5: {additional_data: {main_store_branded_as_local: null, restaurant_branded_as_fresh_kitchen: null},…}

# Code for retrieving Sainsbury's opening times data:

In [ ]:
def get_sainsburys_data(lat,lng):
    API_URL = "https://stores.sainsburys.co.uk/api/v1/stores/"
    params = {
        'fields': 'slfe-list-2.21',
        'api_client_id': 'slfe',
        'store_type': 'main,local',
        'sort': 'by_distance',
        'within': radius,
        'limit': limit,
        'page': '1',
        'lat': lat,
        'lon': lng,
    }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        # print(rq.json())
        return False
    try:
        res = rq.json()
        if res["page_meta"]["total"] == 0:
            return False
        res = res["results"]
    except:
        return False
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res[0]['name'] or res[0]['other_name'])
    #e.g.:
    # while True:
    #     if name != res[0]['name'] or res[0]['other_name']:
    #         i += 1
    #     else:
    #         break
    if res[i]["distance"] > radius * 0.621371:
        return False
    openingHours = res[i]['opening_times']
    dayDoneSet = set()
    daysHeap = []
    for index in range(len(openingHours)):
        key = openingHours[index]['day']
        #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
        if key not in dayDoneSet:
            dayDoneSet.add(key)
            #keyHours processing code
            actualHours = []
            for timeSlotNumber in range(len(openingHours[index]['times'])):
                try:
                    actualHoursDict = {}
                    actualHoursDict['open'] = openingHours[index]['times'][timeSlotNumber]['start_time']
                    actualHoursDict['close'] = openingHours[index]['times'][timeSlotNumber]['end_time']
                    actualHours.append(actualHoursDict)
                except:
                    #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                    closedDayHours = {  'day' : dayKeys[key],
                                    'open' : False  }
                    heapq.heappush(daysHeap, [key,closedDayHours])
                    break
            keyHours = {'day' : dayKeys[key],
                    'open' : True,
                    'hours' : actualHours
                    }
            heapq.heappush(daysHeap, [key,keyHours])
    #Check for any missing days and ensure all days are in order for insertion into our database
    checkedUpToDay = -1
    for day in heapq.nsmallest(7, daysHeap):
        checkedUpToDay += 1
        while day[0] > checkedUpToDay:
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            checkedUpToDay += 1
    while len(daysHeap) < 7:
        if len(daysHeap) > 0:
            checkedUpToDay = daysHeap[-1][0] + 1
        else:
            # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
            checkedUpToDay = 0
        closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
        heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
    if len(daysHeap) != 7:
        return False
    hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
    return hoursArray

# ASDA's API
It is of note that ASDA's system returns a potentially extrememely large quanitity of different qualitative data regarding the store in question, although not all of this data is returned for each store's "response" field. Although, the full schema of possible information for the API to return can be found under "schema" of the API call.

Notes:
Has Click and Collect info.
Diesel price is potentially something that could be displayed (especially if this is displayed on stores like BP Garage etc..).
## Base URL:
https://storelocator.asda.com/index.html
## Parameters:
* q = appears to send off to an external (Google?) API in order to translate whatever form of address you input. Therefore latitude and longitude may be used as so: "latitude,longitude"
* qn = generally seems to mimic what is input into q when used through the website, however when "My Location" button is used, q = lat,lon and qn = "My Location"    **(optional)**
* l = language ("en" for english)  **(optional)**

## Headers:
* accept = "application/json" (otherwise will return html (?))

## Sample API Call:
https://storelocator.asda.com/index.html?q=52.635875,%201.306051&qp=52.635875,%201.306051&l=en
## Sample Response:


In [ ]:
baseUrl: ""
businessId: 2479739
errors: null
featureId: "Search"
isHttps: true
isPrimaryLocale: true
isStaging: false
locale: "en"
queryParams: {l: ["en"], q: ["52.635875, 1.306051"], qp: ["52.635875, 1.306051"]}
response: {count: 2, entities: [{derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…},…],…}
    count: 2
    entities: [{derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…},…]
        0: {derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…}
            derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}}
                address: {countryName: "United Kingdom"}
                    countryName: "United Kingdom"
                uber: {,…}
                    url: "https://m.uber.com/ul/?action=setPickup&client_id=KXQcwoj2Zb8ymDzKgVgbIaDE5iAE_TAj&dropoff%5Bformatted_address%5D=Hall%20Road%2C%20null%2C%20Norwich%2C%20null&dropoff%5Blatitude%5D=52.60555&dropoff%5Blongitude%5D=1.28634&dropoff%5Bnickname%5D=Asda%20Norwich%20Hall%20Road%20Superstore&pickup=my_location"
                    wrappedUrl: "http://a.gotoloc.com/uber/F3EkTqKGX2A"
            distance: {distanceKilometers: 3.4855743361404476, distanceMiles: 2.1658354808794438, id: "4383"}
                distanceKilometers: 3.4855743361404476
                distanceMiles: 2.1658354808794438
                id: "4383"
            listings: {,…}
                googleMyBusiness: {placeId: "ChIJKZs0vAHk2UcRkaLqm7RF3hk", url: "https://maps.google.com/maps?cid=1864003937790239377"}
                    placeId: "ChIJKZs0vAHk2UcRkaLqm7RF3hk"
                    url: "https://maps.google.com/maps?cid=1864003937790239377"
            profile: {,…}
                address: {city: "Norwich", countryCode: "GB", extraDescription: null, line1: "Hall Road", line2: null,…}
                    city: "Norwich"
                    countryCode: "GB"
                    extraDescription: null
                    line1: "Hall Road"
                    line2: null
                    line3: null
                    postalCode: "NR4 6DP"
                    region: null
                    sublocality: null
                c_24HourStore: false
                c_24HourStoreWithSundayClosure: false
                c_aboutSectionCTA: {text: "ASDA.com", url: "https://www.asda.com/"}
                    text: "ASDA.com"
                    url: "https://www.asda.com/"
                c_aboutSectionDescription: "Asda Norwich Hall Road is part of the Asda group; one of Britain’s leading grocery retailers. Asda offers great prices and quality products helping customers save money & live better. Visit us in-store and get groceries delivered to anywhere in the UK, or collect from your local store or petrol station at a convenient time for you."
                c_aboutSectionTitle: "About ASDA - Norwich Hall Road"
                c_alertBannerBackgroundColor: "#5e9b1a"
                c_alertBannerDatePosted: {day: 6, month: 5, year: 2020}
                    day: 6
                    month: 5
                    year: 2020
                c_alertBannerText: "We’re so grateful for the amazing work of our NHS workers, Care Home Workers and Carers, which is why we continue to offer priority access to our stores for these customers <b>between 8am and 9am each Monday, Wednesday and Friday morning—as well as an exclusive hour of browsing on Sunday morning before our stores open.</b>↵↵Please try and avoid coming to our stores at these times to let these vital workers get what they need, and we ask these customers to bring their work ID with them to help our colleagues spot you."
                c_alertBannerTitle: "<b>Priority Shopping</b>"
                c_appleName: "Asda Superstore"
                c_asdaLiving: false
                c_babyChanging: true
                c_bakery: true
                c_bakeryCounter: false
                c_barclaysBank: false
                c_butcherCounter: false
                c_cafe: true
                c_carPark: true
                c_carparkType: ["FREE"]
                    0: "FREE"
                c_cashMachine: true
                c_changingPlacesFacility: false
                c_clickAndCollect: true
                c_clickAndCollectCardTitle: "Click & Collect"
                c_clickAndCollectLockersText: "External Collection Point"
                c_clickAndCollectText: "Click to book your groceries collection at a time to suit you"
                c_communityChamionNominationText1: "How to nominate a group for Green Token Giving"
                c_communityChamionNominationText2: "Email or speak to your Community Champion in store to nominate a cause for Green token giving scheme."
                c_communityChampion: true
                c_communityChampionContactText: "For further information contact your community champion at Community_SouthNorwich@asda.co.uk"
                c_communityChampionNominationLink: "http://www.asda.com"
                c_communityChampionsCTA: {text: "Read More", url: "https://www.asda.com/community-champions"}
                    text: "Read More"
                    url: "https://www.asda.com/community-champions"
                c_communityChampionsImage: {image: {height: 119,…}}
                    image: {height: 119,…}
                        height: 119
                    thumbnails: [{height: 48, url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/196x48.png",…}]
                        0: {height: 48, url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/196x48.png",…}
                            height: 48
                            url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/196x48.png"
                            width: 196
                    url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/480x119.png"
                    width: 480
                c_communityChampionsText: {,…}
                    text(list): ["Local grants", "Green token giving", "Product donation", "Using Asda facilities", "Volunteering"]
                        0: "Local grants"
                        1: "Green token giving"
                        2: "Product donation"
                        3: "Using Asda facilities"
                        4: "Volunteering"
                    text(multi-line): "Are you a charity or not for profit organisation looking for support from Asda? Then contact your Local Community Champion to discuss:"
                c_communityRoom: false
                c_contactSectionTitle: "Contact"
                c_costaCoffee: true
                c_customerWC: true
                c_deliCounter: false
                c_directoryCounty: "East Of England"
                c_disabledFacilities: true
                c_division: "South"
                c_electricCarCharging: true
                c_electricVehicleChargingPoint: true
                c_electricalGoods: true
                c_ethnicFoods: false
                c_expressDiner: true
                c_expressPizzaCounter: false
                c_expressRotisserie: false
                c_facilitiesList: ["Car Park", "Cash Machine", "Customer WC", "Disabled Facilities", "Baby Changing",…]
                    0: "Car Park"
                    1: "Cash Machine"
                    2: "Customer WC"
                    3: "Disabled Facilities"
                    4: "Baby Changing"
                    5: "Electric Vehicle Charging Point"
                    6: "Helium Balloons"
                c_facilitiesTitle: "Facilities"
                c_featuredServiceCardCafe: {hours: {holidayHours: [],…}, photo: {image: {height: 768,…}},…}
                    hours: {holidayHours: [],…}
                        holidayHours: []
                        normalHours: [{day: "MONDAY", intervals: [], isClosed: true}, {day: "TUESDAY", intervals: [], isClosed: true},…]
                            0: {day: "MONDAY", intervals: [], isClosed: true}
                                day: "MONDAY"
                                intervals: []
                                isClosed: true
                            1: {day: "TUESDAY", intervals: [], isClosed: true}
                                day: "TUESDAY"
                                intervals: []
                                isClosed: true
                            2: {day: "WEDNESDAY", intervals: [], isClosed: true}
                                day: "WEDNESDAY"
                                intervals: []
                                isClosed: true
                            3: {day: "THURSDAY", intervals: [], isClosed: true}
                                day: "THURSDAY"
                                intervals: []
                                isClosed: true
                            4: {day: "FRIDAY", intervals: [], isClosed: true}
                                day: "FRIDAY"
                                intervals: []
                                isClosed: true
                            5: {day: "SATURDAY", intervals: [], isClosed: true}
                                day: "SATURDAY"
                                intervals: []
                                isClosed: true
                            6: {day: "SUNDAY", intervals: [], isClosed: true}
                                day: "SUNDAY"
                                intervals: []
                                isClosed: true
                photo: {image: {height: 768,…}}
                    image: {height: 768,…}
                        height: 768
                        sourceUrl: "http://a.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/1152x768.png"
                        thumbnails: [,…]
                            0: {height: 412, url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/619x412.png",…}
                                height: 412
                                url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/619x412.png"
                                width: 619
                            1: {height: 400, url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/600x400.png",…}
                                height: 400
                                url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/600x400.png"
                                width: 600
                            2: {height: 130, url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/196x130.png",…}
                                height: 130
                                url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/196x130.png"
                                width: 196
                        url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/1152x768.png"
                        width: 1152
                    text(multi-line): "Our Asda Cafés have your dining experience covered! We’ve got great food at low prices - just what you’d expect from Asda. What’s more, kids Eat Free Monday-Friday after 3pm when an adult spends just £4, and Pensioners get 10% off all café items on a Wednesday."
                    text(single-line): "Cafe"
                c_firstChoiceTravel: false
                c_fishCounter: false
                c_foodToGo: true
                c_format: "Superstore"
                c_fuelPrice: {diesel: "112.70", effectiveDate: {day: 10, month: 7, year: 2020}, unleaded: "107.70"}
                    diesel: "112.70"
                    effectiveDate: {day: 10, month: 7, year: 2020}
                        day: 10
                        month: 7
                        year: 2020
                    unleaded: "107.70"
                c_george: true
                c_georgeEssentials: false
                c_groceryClickAndCollect: true
                c_groceryClickAndCollectImage: {image: {height: 1080,…}}
                    image: {height: 1080,…}
                        height: 1080
                        sourceUrl: "https://p20.zdusercontent.com/attachment/58910/gDBXr12Pgnpbuo5pjdnenBscP?token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..elcmPhwVI236Uq70DHK-0g.4wX3LoIZBvmeZ8XufZRN95wu_eNx_hQGuZ7Uq7jfwpkhi-7orwPfNKuv8tc5_KFS-UJ3tuiwVRT8HwqIet2PXeT6jXE4w5nHIF7KhoHVBGMu42yauAljxhppLBJb_oFPt89qxHaO5CQNFkaLZsfzV6-i2F26FmglJs0n_34LF4mHOQvASophHAA8SSmIa5rlGUeqrcFd3F1jGjGqq_8UWyOojF9xDgoXqfrZNaqlJCSwC89l7UkLsqtXNAWBLWDswsP7GNYySLzUk5_ut5aAg_DwHDOw2kDiUBJRo35OJuU.MrcEow8y-L3a-zUT-0rR4g"
                        thumbnails: [,…]
                            0: {height: 1068, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/1900x1068.jpg",…}
                                height: 1068
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/1900x1068.jpg"
                                width: 1900
                            1: {height: 348, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/619x348.jpg",…}
                                height: 348
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/619x348.jpg"
                                width: 619
                            2: {height: 337, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/600x337.jpg",…}
                                height: 337
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/600x337.jpg"
                                width: 600
                            3: {height: 110, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/196x110.jpg",…}
                                height: 110
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/196x110.jpg"
                                width: 196
                        url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/1920x1080.jpg"
                        width: 1920
                c_halal: false
                c_homeShopping: true
                c_hotChicken: false
                c_hoursSectionTitle: "Store Hours"
                c_instantPhotoPrint: true
                c_johnsonsDryCleaners: false
                c_linkedEntities: ["4383_C"]
                    0: "4383_C"
                c_maxSpeilman: false
                c_mcdonalds: false
                c_mcgeesButchers: false
                c_mobilePhoneConnectionCentre: false
                c_nearbyStoresSectionTitle: "Nearby Stores"
                c_opticians: false
                c_organic: false
                c_pagesURL: "https://storelocator.asda.com/east-of-england/norwich/hall-road"
                c_paypoint: false
                c_petrolStation: true
                c_pharmacy: false
                c_photoMe: false
                c_pizzaCounter: true
                c_postOffice: false
                c_promoSectionImage: {clickthroughUrl: "https://www.asdagoodliving.co.uk/food/get-your-cook-on",…}
                clickthroughUrl: "https://www.asdagoodliving.co.uk/food/get-your-cook-on"
                    image: {height: 666, thumbnails: [,…],…}
                    height: 666
                    thumbnails: [,…]
                        0: {height: 246, url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/619x246.jpg",…}
                            height: 246
                            url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/619x246.jpg"
                            width: 619
                        1: {height: 239, url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/600x239.jpg",…}
                            height: 239
                            url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/600x239.jpg"
                            width: 600
                        2: {height: 78, url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/196x78.jpg",…}
                            height: 78
                            url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/196x78.jpg"
                            width: 196
                    url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/1670x666.jpg"
                    width: 1670
                c_recycling: false
                c_region: "Anglia"
                c_scanAndGo: true
                c_serviceDetailCardCoinstar: {hours: null, photoField(single): null, text(multi-line): null, text(single-line)1: "Coinstar",…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Coinstar"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardCosta: {hours: null, photoField(single): null, text(multi-line): null, text(single-line)1: "Costa Coffee",…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Costa Coffee"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardGeorge: {hours: null, photoField(single): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): "Visit George to discover clothing, home accessories and footwear for the whole family, including school uniforms, women’s dresses and accessories."
                    text(single-line)1: "George"
                    text(single-line)2: "Shop now"
                    url: "http://www.george.com"
                c_serviceDetailCardGeorgeClickAndCollect: {hours: {holidayHours: [],…}, photoField(single): null,…}
                    hours: {holidayHours: [],…}
                        holidayHours: []
                        normalHours: [{day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false},…]
                            0: {day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "MONDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            1: {day: "TUESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "TUESDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            2: {day: "WEDNESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "WEDNESDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            3: {day: "THURSDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "THURSDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            4: {day: "FRIDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "FRIDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            5: {day: "SATURDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "SATURDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            6: {day: "SUNDAY", intervals: [{end: 1600, start: 1000}], isClosed: false}
                                day: "SUNDAY"
                                intervals: [{end: 1600, start: 1000}]
                                    0: {end: 1600, start: 1000}
                                        end: 1600
                                        start: 1000
                                isClosed: false
                    photoField(single): null
                    text(multi-line): "TO ASSIST OUR STORE COLLEAGUES AT THIS BUSY TIME, THIS SERVICE IS NOT CURRENTLY AVAILABLE"
                    text(single-line)1: "George Click & Collect"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardPetrolFillingStation: {hours: null, photoField(single): null, text(multi-line): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Petrol Filling Station"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardRugDoctor: {hours: null, photoField(single): null, text(multi-line): null, text(single-line)1: "Rug Doctor",…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Rug Doctor"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardScanAndGo: {hours: {holidayHours: [],…}, photoField(single): null,…}
                    hours: {holidayHours: [],…}
                        holidayHours: []
                        normalHours: [{day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false},…]
                            0: {day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "MONDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        1: {day: "TUESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "TUESDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        2: {day: "WEDNESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "WEDNESDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        3: {day: "THURSDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "THURSDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        4: {day: "FRIDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "FRIDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        5: {day: "SATURDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "SATURDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        6: {day: "SUNDAY", intervals: [{end: 1600, start: 1000}], isClosed: false}
                            day: "SUNDAY"
                            intervals: [{end: 1600, start: 1000}]
                                0: {end: 1600, start: 1000}
                                    end: 1600
                                    start: 1000
                            isClosed: false
                    photoField(single): null
                    text(multi-line): "You can use both our Scan & Go handsets and Mobile App in this store."
                    text(single-line)1: "Scan & Go"
                    text(single-line)2: "Read More"
                    url: "https://www.asda.com/about/instore/scan-and-go"
                c_serviceDetailCardToYou: {hours: null, photoField(single): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): "Click/Collect/Return at Asda stores with over a 100 of your favourite retailers."
                    text(single-line)1: "toyou"
                    text(single-line)2: "Read More"
                    url: "https://www.asda.com/toyou/?cmpid=dmc-_-ahc-_-vanityurl-_-generic-_-toyou&utm_source=vanityurl&utm_medium=dmc&utm_term=toyou&utm_content=generic&utm_campaign=ahc"
                c_serviceDetailCardsJustEatKitchen: {hours: null, photoField(single): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): "Hot food for collection & delivery with Just Eat"
                    text(single-line)1: "Just Eat Kitchen"
                    text(single-line)2: null
                    url: null
                c_serviceDetailsTitle: "Services"
                c_storeLocatorFacilitiesFilter: ["ToYou", "Electric Car Charging", "Cash Machine"]
                    0: "ToYou"
                    1: "Electric Car Charging"
                    2: "Cash Machine"
                c_storeLocatorServicesFilter: ["Scan and Go", "Cafe", "Grocery Click & Collect", "Just Eat Kitchen"]
                    0: "Scan and Go"
                    1: "Cafe"
                    2: "Grocery Click & Collect"
                    3: "Just Eat Kitchen"
                c_storeType: ["Superstore"]
                    0: "Superstore"
                c_subway: false
                c_takeawayHotPizza: false
                c_technologyCentre: false
                c_thomasCook: false
                c_timpsons: false
                c_travelMoney: false
                c_travelMoneyClickAndCollect: false
                c_trolleySectionDescription: "Have you found an Asda trolley in your community that has been taken from our store? Let us know exactly where it is and we’ll come and collect it and return it back to Asda."
                c_trolleywise: true
                c_tuiTravel: false
                c_uKRegion: ["England"]
                    0: "England"
                c_whatsAtThisStoreList: ["Rotisserie", "Pizza Counter", "Food To Go", "Instant Photo Print", "Bakery", "Express Diner",…]
                    0: "Rotisserie"
                    1: "Pizza Counter"
                    2: "Food To Go"
                    3: "Instant Photo Print"
                    4: "Bakery"
                    5: "Express Diner"
                    6: "Helium Balloons"
                c_whatsAtThisStoreTitle: "What's at this store"
                cityCoordinate: {lat: 52.63529968261719, long: 1.294010043144226}
                    lat: 52.63529968261719
                    long: 1.294010043144226
                description: "Asda Norwich Hall Road Superstore is part of the Asda group; one of Britain’s leading grocery retailers. Asda offers great prices and quality products helping customers save money & live better. Visit us in-store and get groceries delivered to anywhere in the UK, or collect from your local store or petrol station at a convenient time for you."
                displayCoordinate: {lat: 52.606659, long: 1.288596}
                    lat: 52.606659
                    long: 1.288596
                facebookCoverPhoto: {image: {height: 273,…}}
                    image: {height: 273,…}
                        height: 273
                        sourceUrl: "https://scontent-bos3-1.xx.fbcdn.net/v/t1.0-9/s720x720/96379594_3804414072963302_8747344537808011264_n.png?_nc_cat=1&_nc_sid=dd9801&_nc_ohc=nKEKKHjU_hkAX-q129p&_nc_ht=scontent-bos3-1.xx&oh=ab10e811c6c006d6288d6351d4e576de&oe=5EE042E9"
                        url: "//dynl.mktgcdn.com/p/2aa3EsjnMcrzNB-76pyJ-NkPNOEPPc4tHnk2EYUXytA/720x273.png"
                        width: 720
                facebookName: "Asda"
                facebookPageUrl: "https://www.facebook.com/457762304577879"
                facebookParentPageId: "141700922567987"
                facebookProfilePhoto: {image: {height: 200,…}}
                    image: {height: 200,…}
                        height: 200
                        sourceUrl: "https://scontent-bos3-1.xx.fbcdn.net/v/t31.0-1/p200x200/17359452_1454805254590874_1187074472493250060_o.png?_nc_cat=1&_nc_sid=0c64ff&_nc_ohc=24w1xzbZDC4AX-30l-y&_nc_ht=scontent-bos3-1.xx&oh=4cff6b231ae9659e4873db25995cf638&oe=5EE05BEA"
                        url: "//dynl.mktgcdn.com/p/RB41DA38HIa7bG3QCUXHyDXHx0mdAt8i80htQFuQqos/200x200.png"
                        width: 200
                featuredMessage: {description: "Visit us in-store or online", url: "https://www.asda.com/"}
                    description: "Visit us in-store or online"
                    url: "https://www.asda.com/"
                geocodedCoordinate: {lat: 52.6065825, long: 1.2876707}
                    lat: 52.6065825
                    long: 1.2876707
                geomodifier: "Norwich Hall Road"
                googleAccountId: "117489345900231741701"
                googleAttributes: {}
                googleCoverPhoto: {,…}
                    image: {height: 720, sourceUrl: "https://yext.box.com/shared/static/0oq6t16l8qtmkvbedwbwqt90lm3cv98q.jpg",…}
                        height: 720
                        sourceUrl: "https://yext.box.com/shared/static/0oq6t16l8qtmkvbedwbwqt90lm3cv98q.jpg"
                        thumbnails: [,…]
                            0: {height: 348, url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/619x348.jpg",…}
                                height: 348
                                url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/619x348.jpg"
                                width: 619
                            1: {height: 337, url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/600x337.jpg",…}
                                height: 337
                                url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/600x337.jpg"
                                width: 600
                            2: {height: 110, url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/196x110.jpg",…}
                                height: 110
                                url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/196x110.jpg"
                                width: 196
                            url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/1280x720.jpg"
                            width: 1280
                googlePlaceId: "ChIJKZs0vAHk2UcRkaLqm7RF3hk"
                hours: {holidayHours: [],…}
                    holidayHours: []
                    normalHours: [{day: "MONDAY", intervals: [{end: 2200, start: 700}], isClosed: false},…]
                        0: {day: "MONDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "MONDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        1: {day: "TUESDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "TUESDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        2: {day: "WEDNESDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "WEDNESDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        3: {day: "THURSDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "THURSDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        4: {day: "FRIDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "FRIDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        5: {day: "SATURDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "SATURDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        6: {day: "SUNDAY", intervals: [{end: 1600, start: 1000}], isClosed: false}
                            day: "SUNDAY"
                            intervals: [{end: 1600, start: 1000}]
                                0: {end: 1600, start: 1000}
                                    end: 1600
                                    start: 1000
                            isClosed: false
                isoRegionCode: "NFK"
                logo: {image: {height: 400, thumbnails: [,…],…}}
                    image: {height: 400, thumbnails: [,…],…}
                        height: 400
                        thumbnails: [,…]
                        0: {height: 150, url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/150x150.png",…}
                            height: 150
                            url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/150x150.png"
                            width: 150
                        1: {height: 94, url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/94x94.png",…}
                            height: 94
                            url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/94x94.png"
                            width: 94
                        url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/400x400.png"
                        width: 400
                mainPhone: {countryCode: "GB", display: "01603 964246", number: "+441603964246"}
                    countryCode: "GB"
                    display: "01603 964246"
                    number: "+441603964246"
                matchingSearchIds: ["4656"]
                    0: "4656"
                meta: {accountId: "3332574490752748386", countryCode: "GB", entityType: "location", folderId: "298473",…}
                    accountId: "3332574490752748386"
                    countryCode: "GB"
                    entityType: "location"
                    folderId: "298473"
                    id: "4383"
                    labels: ["78691", "78687", "78686", "78690", "82562", "84251", "95837"]
                        0: "78691"
                        1: "78687"
                        2: "78686"
                        3: "78690"
                        4: "82562"
                        5: "84251"
                        6: "95837"
                    language: "en_GB"
                    schemaTypes: ["GroceryStore", "AutomatedTeller", "GasStation", "ClothingStore"]
                        0: "GroceryStore"
                        1: "AutomatedTeller"
                        2: "GasStation"
                        3: "ClothingStore"
                    timestamp: "2020-07-13T01:47:47"
                    uid: "6wOk6E"
                    utcOffsets: [{offset: 3600, start: 1585443600}, {offset: 0, start: 1603587600}, {offset: 3600, start: 1616893200}]
                        0: {offset: 3600, start: 1585443600}
                            offset: 3600
                            start: 1585443600
                        1: {offset: 0, start: 1603587600}
                            offset: 0
                            start: 1603587600
                        2: {offset: 3600, start: 1616893200}
                            offset: 3600
                            start: 1616893200
                    yextId: 13618640
                name: "Asda Norwich Hall Road Superstore"
                timezone: "Europe/London"
                websiteUrl: "http://storelocator.asda.com/store/norwich-hall-road"
                yextDisplayCoordinate: {lat: 52.6065825, long: 1.2876707}
                lat: 52.6065825
                long: 1.2876707
                yextRoutableCoordinate: {lat: 52.60555, long: 1.28634}
                lat: 52.60555
                long: 1.28634
            url: "east-of-england/norwich/hall-road"
        1: {derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…}
    geo: {address: {city: "Norwich", country: "GB", postalCode: "NR3 1TP", region: "England"},…}
schema: {additionalHoursText: {archived: false, displayName: "Additional Hours Text",…},…}
searchId: "entitysearch"
siteAttributes: {footer: {legal: "© 2019 ASDA. All Rights Reserved.", link1: "Privacy", link2: "Terms of Service",…},…}
siteDomain: "storelocator.asda.com"
siteId: 1812
siteInternalHostName: "storelocator.asda.com"
versions: {en: "index.html"}

# Code for retreiving ASDA's opening hours data:

In [ ]:
def get_asda_data(lat,lng):
    import json
    API_URL = "https://storelocator.asda.com/index.html"
    params = { 'q' : "{0},{1}".format(lat,lng)}
    headers = {
    "accept": "application/json",
  }
    rq = requests.get(API_URL,params=params, headers = headers)
    if rq.status_code != 200:
        print(rq.json())
        return False
    # res = json.dumps(rq.text)
    try:
        res = rq.json()["response"]
        if res["count"] == 0:
            return False
    except:
        return False
    i = 0 
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res["entities"][0]["name"])
    #e.g.:
    # while True:
    #     if name != res["entities"][i]["name"]:
    #         i += 1
    #     else:
    #         break
    if res["entities"][i]["distance"]["distanceKilometers"] > radius:
        return False
    openingHours = res["entities"][i]["profile"]["hours"]["normalHours"]
    hoursArray = []
    intermediaryDayKeys = { 'MONDAY' : 0, 'TUESDAY' : 1, 'WEDNESDAY' : 2, 'THURSDAY' : 3, 'FRIDAY' : 4, 'SATURDAY' : 5, 'SUNDAY' : 6}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    dayDoneSet = set()
    daysHeap = []
    for index in range(len(openingHours)):
            day = intermediaryDayKeys[openingHours[index]["day"]]
            #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
            if day not in dayDoneSet:
                dayDoneSet.add(day)
                if openingHours[index]["isClosed"] != False:
                    closedDayHours = {  'day' : dayKeys[day],
                                        'open' : False  }
                    heapq.heappush(daysHeap, [day,closedDayHours])
                else:
                    actualHours = []
                    for timeSlotNumber in range(len(openingHours[index]['intervals'])):
                        try:
                            actualHoursDict = {}
                            start = str(openingHours[index]['intervals'][timeSlotNumber]['start'])
                            if start == "0":
                                start = "00:00"
                            elif len(start) == 3:
                                start = "0{0}:{1}".format(start[0],start[1:])
                            else:
                                start = "{0}:{1}".format(start[:2],start[2:])
                            end = str(openingHours[index]['intervals'][timeSlotNumber]['end'])
                            if end == "0":
                                end = "00:00"
                            elif len(end) == 3:
                                end = "0{0}:{1}".format(end[0],end[1:])
                            else:
                                end = "{0}:{1}".format(end[:2],end[2:])
                            actualHoursDict['open'] = start
                            actualHoursDict['close'] = end
                            actualHours.append(actualHoursDict)
                        except:
                            #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                            closedDayHours = {  'day' : dayKeys[day],
                                            'open' : False  }
                            heapq.heappush(daysHeap, [day,closedDayHours])
                            break
                    keyHours = {'day' : dayKeys[day],
                            'open' : True,
                            'hours' : actualHours
                            }
                    heapq.heappush(daysHeap, [day,keyHours])

    #Check for any missing days and ensure all days are in order for insertion into our database     
    checkedUpToDay = -1
    for day in heapq.nsmallest(7, daysHeap):
        checkedUpToDay += 1
        while day[0] > checkedUpToDay:
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            checkedUpToDay += 1
    while len(daysHeap) < 7:
        if len(daysHeap) > 0:
            checkedUpToDay = daysHeap[-1][0] + 1
        else:
            # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
            checkedUpToDay = 0
        closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
        heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
    if len(daysHeap) != 7:
        return False
    hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
    return hoursArray

# Tesco's API

## Parameters:
* offset : 0 
* limit : number of stores to show (int)
* sort : "near:%22{LATITUDE},{LONGITUDE}%22"
* filter : "category:Store%20AND%20isoCountryCode:x-uk"
* fields : what to return (subset of <code>"name,geo,openingHours,altIds.branchNumber,contact,facilities"</code>)

## Headers:
* x-appkey : "store-locator-web-cde"

 
## Sample API Call directly from website:
<code>rq = requests.get("https://api.tesco.com/tescolocation/v3/locations/search?offset=0&limit=10&sort=near:%2252.62682342529297,1.3366719484329224%22&filter=category:Store%20AND%20isoCountryCode:x-uk&fields=name,geo,openingHours,altIds.branchNumber,contact,facilities", headers={"x-appkey": "store-locator-web-cde"})
</code>

RESULTS:
* of interest is: rq.json()['results'][0]['location']['openingHours'][0]['standardOpeningHours']
* and : rq.json()['results'][0]['location']['openingHours'][0]['exceptions']
* However, exceptions provides you a historical(?) list of dictionaries of all dates that have had exceptions in the past(?), in a seemingly completely unsorted order, making using this information for anything other than extrapolation (**NOT IMPLEMENTED**) of 'special dates'' opening times (e.g. christmas) somewhat unfeasible.

### Sample response:

In [ ]:
{,…}
    results: [{location: {id: "2ad4c578-962b-42cd-8541-0f75c1952680", name: "Norwich Plumstead Rd Express",…},…},…]
        0: {location: {id: "2ad4c578-962b-42cd-8541-0f75c1952680", name: "Norwich Plumstead Rd Express",…},…}
            distanceFrom: {unit: "miles", value: 0.8}
                unit: "miles"
                value: 0.8
            location: {id: "2ad4c578-962b-42cd-8541-0f75c1952680", name: "Norwich Plumstead Rd Express",…}
                altIds: {branchNumber: 5284}
                    branchNumber: 5284
                contact: {address: {lines: [{lineNumber: 1, text: "197 Plumstead Rd"}], town: "Norwich", postcode: "NR1 4AB"},…}
                    address: {lines: [{lineNumber: 1, text: "197 Plumstead Rd"}], town: "Norwich", postcode: "NR1 4AB"}
                        lines: [{lineNumber: 1, text: "197 Plumstead Rd"}]
                            0: {lineNumber: 1, text: "197 Plumstead Rd"}
                                lineNumber: 1
                                text: "197 Plumstead Rd"
                            postcode: "NR1 4AB"
                            town: "Norwich"
                    phoneNumbers: [{alias: "Default", number: "0345 674 6492"}]
                        0: {alias: "Default", number: "0345 674 6492"}
                            alias: "Default"
                            number: "0345 674 6492"
                facilities: [{name: "FREE_FROM", tags: ["public", "food_range"], description: "Free From"},…]
                0: {name: "FREE_FROM", tags: ["public", "food_range"], description: "Free From"}
                    description: "Free From"
                    name: "FREE_FROM"
                    tags: ["public", "food_range"]
                        0: "public"
                        1: "food_range"
                1: {name: "ATM", tags: ["public"], description: "ATM"}
                    description: "ATM"
                    name: "ATM"
                    tags: ["public"]
                        0: "public"
                2: {name: "GIFT_CARDS", tags: ["public"], description: "Sells Gift Cards"}
                    description: "Sells Gift Cards"
                    name: "GIFT_CARDS"
                    tags: ["public"]
                        0: "public"
                3: {name: "TESCO_PAY_PLUS", tags: ["public"], description: "Tesco Pay+"}
                    description: "Tesco Pay+"
                    name: "TESCO_PAY_PLUS"
                    tags: ["public"]
                        0: "public"
                4: {name: "ASSISTANCE_DOGS", tags: ["enabling", "public"],…}
                    description: "Assistance dogs are welcome in our store."
                    name: "ASSISTANCE_DOGS"
                    tags: ["enabling", "public"]
                        0: "enabling"
                        1: "public"
                5: {name: "PAYPOINT", tags: ["public"], description: "PayPoint"}
                    description: "PayPoint"
                    name: "PAYPOINT"
                    tags: ["public"]
                        0: "public"
                geo: {coordinates: {longitude: 1.32715, latitude: 52.63697}}
                    coordinates: {longitude: 1.32715, latitude: 52.63697}
                        latitude: 52.63697
                        longitude: 1.32715
                id: "2ad4c578-962b-42cd-8541-0f75c1952680"
                name: "Norwich Plumstead Rd Express"
                openingHours: [{type: "Trading",…}]
                    0: {type: "Trading",…}
                        exceptions: [{date: "2017-12-25", hours: {isOpen: "false"}},…]
                            0: {date: "2017-12-25", hours: {isOpen: "false"}}
                                date: "2017-12-25"
                                hours: {isOpen: "false"}
                                    isOpen: "false"
                            1: {date: "2020-03-21", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-21"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            2: {date: "2017-12-26", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2017-12-26"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            3: {date: "2020-03-22", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-22"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            4: {date: "2020-03-18", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-18"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            5: {date: "2019-12-31", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2019-12-31"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            6: {date: "2018-12-24", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2018-12-24"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            7: {date: "2020-03-23", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-23"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            8: {date: "2019-12-25", hours: {isOpen: "false"}}
                                date: "2019-12-25"
                                hours: {isOpen: "false"}
                                    isOpen: "false"
                            9: {date: "2020-03-19", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-19"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            10: {date: "2018-01-01", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2018-01-01"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            11: {date: "2018-12-31", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2018-12-31"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            12: {date: "2019-12-26", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2019-12-26"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            13: {date: "2020-01-01", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2020-01-01"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            14: {date: "2018-12-25", hours: {isOpen: "false"}}
                                date: "2018-12-25"
                                hours: {isOpen: "false"}
                                    isOpen: "false"
                            15: {date: "2017-12-24", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2017-12-24"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            16: {date: "2019-12-24", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2019-12-24"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            17: {date: "2017-12-31", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2017-12-31"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            18: {date: "2019-01-01", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2019-01-01"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            19: {date: "2020-03-20", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-20"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            20: {date: "2018-12-26", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2018-12-26"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                        standardOpeningHours: {mo: {isOpen: "true", open: "0600", close: "2300"}, tu: {isOpen: "true", open: "0600", close: "2300"},…}
                            fr: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            mo: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            sa: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            su: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            th: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            tu: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            we: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                        type: "Trading"
        1: {location: {id: "122a682a-34df-45f3-92d7-0cd3c4326067", name: "Norwich Wales Rd Express",…},…}
        2: {location: {id: "8c7b5bce-0139-4831-8f58-7280d88f6ed4", name: "Norwich Westlegate Express",…},…}
    
    

 # Code for retrieving Tesco's opening times data:

In [ ]:
def get_tesco_data(lat, lng):
    API_URL = "https://api.tesco.com/tescolocation/v3/locations/search"
    params = {  'offset' : 0,
                'limit' : limit,
                'sort' : 'near:"{0},{1}"'.format(lat,lng),
                'filter' : "category:Store AND isoCountryCode:x-uk",
                'fields' : "name,geo,openingHours"
                #known fields: "name,geo,openingHours,altIds.branchNumber,contact,facilities"
             }
    headers = {"x-appkey": "store-locator-web-cde"}
    rq = requests.get(API_URL, params=params, headers=headers)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()['results']
        if len(res) == 0:
            return False
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res[0]['location']['name'])
    #e.g.:
    # while True:
    #     if name != res[i]['location']['name']:
    #         i += 1
    #     else:
    #         break
    if res[i]["distanceFrom"]["value"] > 0.621371*radius:
        return False
    openingHours = res[i]['location']['openingHours'][0]['standardOpeningHours']
    dayKeys = { 'mo' : 'Monday', 'tu' : 'Tuesday', 'we' : 'Wednesday', 'th' : 'Thursday', 'fr' : 'Friday', 'sa' : 'Saturday', 'su' : 'Sunday'}
    hoursArray = []
    for key in ['mo','tu','we','th','fr','sa','su']:
        try:
            if openingHours[key]['isOpen'] == 'true':
                actualHours = {'open' : openingHours[key]['open'][:2] + ":" + openingHours[key]['open'][2:],
                            'close' : openingHours[key]['close'][:2] + ":" + openingHours[key]['close'][2:]}
                keyHours = {'day' : dayKeys[key],
                        'open' : True,
                        'hours' : [actualHours]
                        }
            else:
                keyHours = {'day' : dayKeys[key],
                        'open' : False}
        except:
            keyHours = {'day' : dayKeys[key],
                        'open' : False}

        hoursArray.append(keyHours)
    return hoursArray


# Morrison's API
## Base URL:
https://api.morrisons.com/location/v2//stores
## Parameters:
* distance = radius within which to search for store, measured in km (**tested**), website uses 50000 and returns at least 21 miles radius, so one assumes meters, so 50km radius.
* lat = latitude
* lon = longitude
* limit = number of stores to display
* offset = (?) wesbite uses 0
* storeformat = "supermarket"

## Headers:
* apikey = one i used on first day: "kxBdM2chFwZjNvG2PwnSn3sj6C53dLEY" (appears to still work 2 days later, but still same location (**Note: further testing required**))

## Sample API Call:
<code>https://api.morrisons.com/location/v2//stores?apikey=kxBdM2chFwZjNvG2PwnSn3sj6C53dLEY&distance=50000&lat=52.62682342529297&limit=10&lon=1.3366719484329224&offset=0&storeformat=supermarket</code>
### Sample Response:

In [1]:
{,…}
    metadata: {limit: 10, count: 10, offset: 0, total: 12}
    stores: [{name: 115, storeName: "Norwich", storeFormat: "supermarket", category: "Supermarket",…},…]
        0: {name: 115, storeName: "Norwich", storeFormat: "supermarket", category: "Supermarket",…}
            address: {addressLine1: "4 Albion Way", addressLine2: "Riverside", county: "Norfolk", city: "Norwich",…}
                addressLine1: "4 Albion Way"
                addressLine2: "Riverside"
                city: "Norwich"
                country: "United Kingdom"
                county: "Norfolk"
                postcode: "NR1 1WR"
            category: "Supermarket"
            distance: 2065
            location: {latitude: 52.623662, longitude: 1.30663}
                latitude: 52.623662
                longitude: 1.30663
            name: 115
            openingTimes: {mon: {open: "07:00:00", close: "22:00:00"}, tue: {open: "07:00:00", close: "22:00:00"},…}
                fri: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                mon: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                sat: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                sun: {open: "10:00:00", close: "16:00:00"}
                    close: "16:00:00"
                    open: "10:00:00"
                thu: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                tue: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                wed: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
            region: "East England"
            satnav: {latitude: 52.623662, longitude: 1.30663}
                latitude: 52.623662
                longitude: 1.30663
            storeFormat: "supermarket"
            storeName: "Norwich"
            telephone: "01603 663999"
        1: {name: 385, storeName: "Catton", storeFormat: "supermarket", category: "Supermarket",…}
        2: {name: 655, storeName: "Wymondham", storeFormat: "supermarket", category: "Supermarket",…}
        3: {name: 275, storeName: "Beccles", storeFormat: "supermarket", category: "Supermarket",…}


SyntaxError: invalid syntax (<ipython-input-1-3db5c2fcad70>, line 1)

# Code for retrieving Morrison's opening hours data:

In [ ]:
def get_morrisons_data(lat, lng):
    apikey = "kxBdM2chFwZjNvG2PwnSn3sj6C53dLEY"
    API_URL = "https://api.morrisons.com/location/v2//stores"
    params = { 'apikey' : apikey,
                'distance' : radius*1000,
                'lat' : lat,
                'lon' : lng,
                'limit' : limit,
                'offset' : 0,
                'storeformat' : "supermarket"
             }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()["stores"]
        if len(res) == 0:
            return False
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. "Morrisons" + res[0]["storename"])
    #e.g.:
    # while True:
    #     if name != "Morrisons" + res[i]["storename"]:
    #         i += 1
    #     else:
    #         break
    if res[i]["distance"] > radius * 1000:
        return False 
    openingHours = res[i]['openingTimes']
    dayKeys = { 'mon' : 'Monday', 'tue' : 'Tuesday', 'wed' : 'Wednesday', 'thu' : 'Thursday', 'fri' : 'Friday', 'sat' : 'Saturday', 'sun' : 'Sunday'}
    hoursArray = []
    for key in ['mon','tue','wed','thu','fri','sat','sun']:
        try:
            actualHours = {'open' : openingHours[key]['open'][:5],
                        'close' : openingHours[key]['close'][:5]}
            keyHours = {'day' : dayKeys[key],
                    'open' : True,
                    'hours' : [actualHours]
                    }
        except:
            keyHours = {'day' : dayKeys[key],
                        'open' : False}

        hoursArray.append(keyHours)
    return hoursArray


# Waitrose's "API":
Waitrose's system works very differently from the majority of other supermarkets'.
Waitrose first calls their NearestBranchesCmd API, to obtain a list of the 6 nearest branches.
Waitrose then calls their StandardWorkingHoursView API for the closest 3 of those stores, which returns an html encoded table(?) of the opening hours of that branch (see examples below).
## NearestBranchesCmd API:
### Base URL:
https://www.waitrose.com/shop/NearestBranchesCmd
### Parameters:
* latitude = latitude
* longitude = longitude
* fromMultipleBranch = True (Note: True/False doesnt seem to make a difference) (optional)
* _ = 1594910860791 (on the first day i used it, perhaps this is some form of API key??) (optional)
### Sample API Call:
https://www.waitrose.com/shop/NearestBranchesCmd?latitude=52.6054482&longitude=1.2412015&fromMultipleBranch=true&_=1594910860791
https://www.waitrose.com/shop/NearestBranchesCmd?latitude=52.6470831&longitude=1.3433364&fromMultipleBranch=true&_=1594911360288
### Sample Response:

In [ ]:
{success: true, branchList: [{branchId: 222, branchName: "Norwich", phoneNumber: "01603 458114",…},…]}
    branchList: [{branchId: 222, branchName: "Norwich", phoneNumber: "01603 458114",…},…]
        0: {branchId: 222, branchName: "Norwich", phoneNumber: "01603 458114",…}
            addressLine1: "Eaton Centre, Church Lane"
            branchDesc: "Standard Branch"
            branchId: 222
            branchName: "Norwich"
            city: "Eaton"
            isEntertaining: false
            isGrocery: false
            latitude: 52.60794
            longitude: 1.25119
            phoneNumber: "01603 458114"
            postCode: "NR4 6NU"
        1: {branchId: 662, branchName: "Wymondham", phoneNumber: "01953 607772", addressLine1: "Norwich Road",…}
        2: {branchId: 622, branchName: "North Walsham", phoneNumber: "01692 503109", addressLine1: "Cromer Road",…}
        3: {branchId: 686, branchName: "Swaffham", phoneNumber: "01760 336307", addressLine1: "Castle Acre Road",…}
        4: {branchId: 483, branchName: "Saxmundham", phoneNumber: "01728 603004", addressLine1: "Church Street",…}
        5: {branchId: 140, branchName: "Bury St Edmunds", phoneNumber: "01284 725222",…}
    success: true

## StandardWorkingHours API:
### Base URL:
https://www.waitrose.com/shop/StandardWorkingHoursView
### Parameters:
* branchId = id of branch (retrieved from the NearestBranchesCmd API)
* storeId = id of store (retrieved from the NearestBranchesCmd API)
* linkType = "standard" (not sure what other options could be)
* _ = 1594911360644 (on first try, once again unsure of what this is or does, but does seem to go up over quite quickly time, so perhaps a global counter?)

### Sample API Call:
https://www.waitrose.com/shop/StandardWorkingHoursView?branchId=222&storeId=10317&linkType=standard&_=1594911360644
### Sample Response:
#### Note:
Edit this cell in order to see exact response received (the < pre > and < /pre > tags have been added in for formatting and were not part of the original reponse).
<pre>

<table>
<tbody>
	
			 <tr>
			 
			 <td class="day">
			 Sunday:
			 </td>
 		      
			  <td class="time">
			    10:00 - 16:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Monday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Tuesday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Wednesday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Thursday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Friday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Saturday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
</tbody>
</table>

</pre>

### Version obtained in plaintext via opening in Chrome (although in browser this does display with newlines):
Sunday:	10:00 - 16:00
Monday:	07:00 - 20:00
Tuesday:	07:00 - 20:00
Wednesday:	07:00 - 20:00
Thursday:	07:00 - 20:00
Friday:	07:00 - 20:00
Saturday:	07:00 - 20:00

# Code for retreiving Waitrose's opening hours data:

In [ ]:
def get_waitrose_data(lat, lng):
    API_URL_locations = "https://www.waitrose.com/shop/NearestBranchesCmd"
    API_URL_hours = "https://www.waitrose.com/shop/StandardWorkingHoursView"
    params_locations = {'latitude' : lat,
                        'longitude' : lng,
                        }
    rq = requests.get(API_URL_locations, params=params_locations)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()
        if not res['success'] or len(res['branchList']) == 0:
            return False
        res = res["branchList"]
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. "Waitrose" + res[0]["branchName"])
    #e.g.:
    # while True:
    #     if name != "Waitrose" + res[i]["branchName"]:
    #         i += 1
    #     else:
    #         break
    storeLat, storeLng = res[i]["latitude"], res[i]["longitude"]
    #compute distance between the two points using the haversine function
    storeDistance = haversine((lat, lng),(storeLat, storeLng))
    if storeDistance > radius:
        return False
    branchId = res[i]['branchId']
    params_hours = { 'branchId' : branchId}
    rq = requests.get(API_URL_hours, params=params_hours)
    hoursStringHTML = rq.text
    cleanr = re.compile('<.*?>|\s')
    hoursString = re.sub(cleanr, '', hoursStringHTML)
    #This section is to ensure if the shop is closed, correct times are displayed
    sundayIndex = hoursString.find("Sunday:")
    mondayIndex = hoursString.find("Monday:")
    tuesdayIndex = hoursString.find("Tuesday:")
    wednesdayIndex = hoursString.find("Wednesday:")
    thursdayIndex = hoursString.find("Thursday:")
    fridayIndex = hoursString.find("Friday:")
    saturdayIndex = hoursString.find("Saturday:")

    hoursArray = []
    #equivalent of the usual for loop V
    if mondayIndex == -1:
        keyHours = {'day' : "Monday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[mondayIndex+7:tuesdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Monday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Monday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Monday",
                        'open' : False}
            hoursArray.append(keyHours)
            

    if tuesdayIndex == -1:
        keyHours = {'day' : "Tuesday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[tuesdayIndex+8:wednesdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Tuesday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Tuesday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Tuesday",
                        'open' : False}
            hoursArray.append(keyHours)
    if wednesdayIndex == -1:
        keyHours = {'day' : "Wednesday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[wednesdayIndex+10:thursdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Wednesday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Wednesday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Wednesday",
                        'open' : False}
            hoursArray.append(keyHours)
        
    if thursdayIndex == -1:
        keyHours = {'day' : "Thursday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[thursdayIndex+9:fridayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Thursday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Thursday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Thursday",
                        'open' : False}
            hoursArray.append(keyHours)
    if fridayIndex == -1:
        keyHours = {'day' : "Friday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[fridayIndex+7:saturdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Friday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Friday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Friday",
                        'open' : False}
            hoursArray.append(keyHours)
    if saturdayIndex == -1:
        keyHours = {'day' : "Saturday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[saturdayIndex+9:]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Saturday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Saturday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Saturday",
                        'open' : False}
            hoursArray.append(keyHours)
    if sundayIndex == -1:
        keyHours = {'day' : "Sunday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[sundayIndex+7:mondayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Sunday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Sunday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Sunday",
                        'open' : False}
            hoursArray.append(keyHours)
    return hoursArray


# Aldi's API:
## Base URL:
https://www.aldi.co.uk/api/store-finder/search
## Parameters:
* q = postcode (optional)
* address = Road, Postcode (optional)
* latitude = latidude
* longitude = longitude

## Example API call:
https://www.aldi.co.uk/api/store-finder/search?q=nr70en&address=Yarmouth+Rd%2C+Norwich+NR7+0EN&latitude=52.6266168&longitude=1.3366618
## Example response:

In [ ]:
{query: "nr70en", address: "Yarmouth Rd, Norwich NR7 0EN",…}
    address: "Yarmouth Rd, Norwich NR7 0EN"
    bounds: {n: "52.64952", e: "1.4334135999999997", s: "52.6037136", w: "1.23991"}
        e: "1.4334135999999997"
        n: "52.64952"
        s: "52.6037136"
        w: "1.23991"
    center: {lat: "52.6266168", lng: "1.3366618"}
        lat: "52.6266168"
        lng: "1.3366618"
    formattedCount: "Nearest <strong>30 stores</strong> found for <strong>Yarmouth Rd, Norwich NR7 0EN</strong>"
    pagination: {next: 1,…}
        next: 1
        pages: [{page: 0, prettyPage: 1, currentPage: true, center: {lat: "52.6266168", lng: "1.3366618"},…},…]
            0: {page: 0, prettyPage: 1, currentPage: true, center: {lat: "52.6266168", lng: "1.3366618"},…}
                address: "Yarmouth Rd, Norwich NR7 0EN"
                center: {lat: "52.6266168", lng: "1.3366618"}
                    lat: "52.6266168"
                    lng: "1.3366618"
                currentPage: true
                page: 0
                prettyPage: 1
                query: "nr70en"
            1: {page: 1, prettyPage: 2, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            2: {page: 2, prettyPage: 3, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            3: {page: 3, prettyPage: 4, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            4: {page: 4, prettyPage: 5, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            5: {page: 5, prettyPage: 6, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
    query: "nr70en"
    results: [,…]
        0: {code: "s-uk-E0566", name: "ALDI - 174-178 Plumstead Road", distance: "0.8 Miles", description: "",…}
            address: ["Norwich", "NR1 4JZ"]
                0: "Norwich"
                1: "NR1 4JZ"
            code: "s-uk-E0566"
            description: ""
            distance: "0.8 Miles"
            features: ["parking"]
                0: "parking"
            isFirstStore: true
            latlng: {lat: 52.6366, lng: 1.32667}
                lat: 52.6366
                lng: 1.32667
            name: "ALDI - 174-178 Plumstead Road"
            openingTimes: [{day: "Mon", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false},…]
                0: {day: "Mon", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Mon"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                1: {day: "Tue", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Tue"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                2: {day: "Wed", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Wed"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                3: {day: "Thu", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Thu"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                4: {day: "Fri", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Fri"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                5: {day: "Sat", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Sat"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                6: {day: "Sun", hours: "10:00&nbsp;&ndash;&nbsp;16:00", closed: false, special: false}
                    closed: false
                    day: "Sun"
                    hours: "10:00&nbsp;&ndash;&nbsp;16:00"
                    special: false
            selectedPage: "0"
        1: {code: "s-uk-E0487", name: "ALDI - 463 Sprowston Road", distance: "2 Miles", description: "",…}
        2: {code: "s-uk-E1022", name: "ALDI - Unit 5, Hall Road", distance: "2.3 Miles", description: "",…}
        3: {code: "s-uk-E1091", name: "ALDI - Unit1, 36 - 50 Drayton Road", distance: "2.6 Miles",…}
        4: {code: "s-uk-E0377", name: "ALDI - 1 Larkman Lane", distance: "4.1 Miles", description: "",…}

# Code for retreiving Aldi's opening hours data:

In [ ]:
def get_aldi_data(lat, lng):
    API_URL = "https://www.aldi.co.uk/api/store-finder/search"
    params = {  'latitude' : lat,
                'longitude' : lng,
                'fromMultipleBranch' : False
             }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()["results"]
        if len(res) == 0:
            return False
    except:
        return False
    i=0
    intermediaryDayKeys = { 'Mon' : 0, 'Tue' : 1, 'Wed' : 2, 'Thu' : 3, 'Fri' : 4, 'Sat' : 5, 'Sun' : 6}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0]["name"])
    #e.g.:
    # while True:
    #     if name != res[i]["name"]:
    #         i += 1
    #     else:
    #         break
    distance = float(res[i]["distance"].split(" ")[0]) * 1.60934
    if distance > radius:
        return False
    openingHours = res[i]["openingTimes"]
    hoursArray = []
    dayDoneSet = set()
    daysHeap = []
    for index in range(len(openingHours)):
            day = intermediaryDayKeys[openingHours[index]["day"]]
            #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
            if day not in dayDoneSet:
                dayDoneSet.add(day)
                #keyHours processing code
                key = intermediaryDayKeys[openingHours[index]['day']]  #turn key into int
                actualHours = []
                if openingHours[index]['closed'] == False:
                    hoursString = html.unescape(openingHours[index]['hours'])
                    while len(hoursString) > 11: #incase there are multiple time slots encoded within the string (format is unknown hence the strange code)
                        timeSlot = { 'open' : hoursString[:5],
                                        'close' : hoursString[8:13]}
                        actualHours.append(timeSlot)
                        hoursString = hoursString[13:]
                        if len(hoursString) > 0:
                            try:
                                firstDigitIndex = [x.isdigit() for x in hoursString].index(True)
                                hoursString = hoursString[firstDigitIndex:]
                            except:
                                break
                    keyHours = {'day' : dayKeys[key],
                        'open' : True,
                        'hours' : actualHours
                        }
                else:
                    keyHours = {'day' : dayKeys[key],
                        'open' : False
                        }
                heapq.heappush(daysHeap, [key, keyHours])
    #Check for any missing days and ensure all days are in order for insertion into our database
    checkedUpToDay = -1
    for day in heapq.nsmallest(7, daysHeap):
        checkedUpToDay += 1
        while day[0] > checkedUpToDay:
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            checkedUpToDay += 1
    while len(daysHeap) < 7:
        if len(daysHeap) > 0:
            checkedUpToDay = daysHeap[-1][0] + 1
        else:
            # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
            checkedUpToDay = 0
        closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
        heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
    if len(daysHeap) != 7:
        return False
    hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
    return hoursArray


# Co-op's API
## Base URL:
https://api.coop.co.uk/locationservices/finder/food/
## Parameters:
* location = "Latitude,Longitude"
* distance = maximum distance to look for stores within (m)
* min_distance = minimum distance to look for stores within (m)
* min_results = amount of results that must (at least) be returned, if not radius is increased until a store is found(?) ( **WARNING:** if this occurs, a bug in their API occurs and day indexing in the response is reversed with sunday being 0 and monday 6, therefore setting min_results = 0 is recommended.
(**Note:** from version 9 of this project onwards, after the redesign of various functions using heaps instead, this no longer causes an issue (as heaps were implemented specifically to deal with out of order days better while maintaining the array format of hoursArray)))
* format = "json"
## Sample API Call:
https://api.coop.co.uk/locationservices/finder/food/?location=52.6470831%2C1.3433364&distance=3000&min_distance=0&min_results=1&format=json
## Sample Response:

In [ ]:
count: 4
next: null
previous: null
results: [{name: "Heartsease", location_type: "Food", society: "East of England Co-operative Society",…},…]
    0: {name: "Heartsease", location_type: "Food", society: "East of England Co-operative Society",…}
        amazon_counter_shortname: ""
        anacode: "5000128514185"
        cedar: ""
        county: "Norfolk"
        distance: {m: 810}
            m: 810
        email: null
        facilities: []
        hubnumber: null
        image: null
        location_type: "Food"
        name: "Heartsease"
        open_status: "Open"
        open_status_label: "Open until 10pm"
        opening_hours: [{name: "Monday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"},…]
            0: {name: "Monday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Monday"
                opens: "07:00"
                type: "hours"
            1: {name: "Tuesday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Tuesday"
                opens: "07:00"
                type: "hours"
            2: {name: "Wednesday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Wednesday"
                opens: "07:00"
                type: "hours"
            3: {name: "Thursday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Thursday"
                opens: "07:00"
                type: "hours"
            4: {name: "Friday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Friday"
                opens: "07:00"
                type: "hours"
            5: {name: "Saturday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Saturday"
                opens: "07:00"
                type: "hours"
            6: {name: "Sunday", label: "7am to 10pm", type: "hours", opens: "07:00", closes: "22:00"}
                closes: "22:00"
                label: "7am to 10pm"
                name: "Sunday"
                opens: "07:00"
                type: "hours"
        phone: "01603 433127"
        position: {x: 1.33706, y: 52.64088, srid: 4326}
            srid: 4326
            x: 1.33706
            y: 52.64088
        postcode: "NR7 9DX"
        public: true
        seasonal_hours: null
        society: "East of England Co-operative Society"
        street_address: "Witard Road"
        street_address2: "Heartease Estate"
        street_address3: ""
        temporarily_closed: false
        town: "Norwich"
        url: "/food/store/NR7-9DX/witard-road"
        website: ""
1: {name: "Thorpe St Andrew", location_type: "Food", society: "East of England Co-operative Society",…}
2: {name: "Cannerby Lane", location_type: "Food", society: "East of England Co-operative Society",…}
3: {name: "Norwich - Sprowston Road", location_type: "Food", society: "The Co-operative Group",…}

# Code for retrieving Co-op's opening hours data:

In [ ]:
def get_coop_data(lat, lng):
    API_URL = "https://api.coop.co.uk/locationservices/finder/food/"
    params = {  'location' : "{0},{1}".format(lat,lng),
                'distance' : radius*1000,
                'min_distance' : "0",
                'min_results' : "0",
                'format' : "json"}
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()
        if res['count'] == 0:
            return False
        res = res["results"]
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0]["name"])
    #e.g.:
    # while True:
    #     if name != res[i]["name"]:
    #         i += 1
    #     else:
    #         break
    if res[i]["distance"]["m"] > radius * 1000:
        return False
    openingHours = res[i]["opening_hours"]
    intermediaryDayKeys = { 'Monday' : 0, 'Tuesday' : 1, 'Wednesday' : 2, 'Thursday' : 3, 'Friday' : 4, 'Saturday' : 5, 'Sunday' : 6}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    dayDoneSet = set()
    daysHeap = []
    for index in range(len(openingHours)):
            key = intermediaryDayKeys[openingHours[index]['name']]  #turn key into int
            #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
            if key not in dayDoneSet:
                dayDoneSet.add(key)
                #keyHours processing code
                actualHours = []
                try:
                    if openingHours[index]["label"].find("los") != -1:
                        #incase the world closed/closure/Closed/Closure is in the label
                        closedDayHours = {  'day' : dayKeys[key],
                                    'open' : False  }
                        heapq.heappush(daysHeap, [key, closedDayHours])
                    else:
                        actualHoursDict = {}
                        actualHoursDict['open'] = openingHours[index]['opens']
                        actualHoursDict['close'] = openingHours[index]['closes']
                        actualHours = [actualHoursDict]
                        keyHours = {'day' : dayKeys[key],
                            'open' : True,
                            'hours' : actualHours
                            }
                        heapq.heappush(daysHeap, [key, keyHours])
                except:
                    #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                    closedDayHours = {  'day' : dayKeys[key],
                                    'open' : False  }
                    heapq.heappush(daysHeap, [key, closedDayHours])
    # Check for any missing days and ensure all days are in order for insertion into our database
    checkedUpToDay = -1
    for day in heapq.nsmallest(7, daysHeap):
        checkedUpToDay += 1
        while day[0] > checkedUpToDay:
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            checkedUpToDay += 1
    while len(daysHeap) < 7:
        if len(daysHeap) > 0:
            checkedUpToDay = daysHeap[-1][0] + 1
        else:
            # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
            checkedUpToDay = 0
        closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
        heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
    if len(daysHeap) != 7:
        return False
    hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
    return hoursArray


# Marks and Spencer's API
## Base URL:
https://api.marksandspencer.com/v1/stores

## Parameters:
* apikey = "aVCi8dmPbHgHrdCv9gNt6rusFK98VokK" on the first day (**required**)
* jsonp = Appears to put a plaintext prefix before the json response. Website uses "angular.callbacks._1" (optional)
* latlong = "Latitude,Longitude"
* limit = number of stores to return
* radius = radius within which to look for stores in (miles or km)

## Sample API Call:
https://api.marksandspencer.com/v1/stores?apikey=aVCi8dmPbHgHrdCv9gNt6rusFK98VokK&jsonp=angular.callbacks._1&latlong=52.64497375488281,1.2892669439315796&limit=10&radius=40
## Sample Response:
**Note:** the 0'th store is a BP Garage, and the 2nd store is a more usual M&S store, therefore the 2 results are both displayed.

In [ ]:
angular.callbacks._1({results: [{id: 1977, name: "MOUSEHOLD BP", storeType: "franchise",…},…], count: 10});
    count: 10
    results: [{id: 1977, name: "MOUSEHOLD BP", storeType: "franchise",…},…]
        0: {id: 1977, name: "MOUSEHOLD BP", storeType: "franchise",…}
            address: {addressLine1: "MOUSEHOLD BP", addressLine2: "58 Mousehold Lane", city: "NORWICH",…}
                addressLine1: "MOUSEHOLD BP"
                addressLine2: "58 Mousehold Lane"
                city: "NORWICH"
                country: "United Kingdom"
                isoTwoCountryCode: "GB"
                postalCode: "NR7 8HA"
            coordinates: {latitude: 52.648823, longitude: 1.31477}
                latitude: 52.648823
                longitude: 1.31477
            coreOpeningHours: [{day: "Monday", open: "00:00", close: "24:00"}, {day: "Tuesday", open: "00:00", close: "24:00"},…]
                0: {day: "Monday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Monday"
                    open: "00:00"
                1: {day: "Tuesday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Tuesday"
                    open: "00:00"
                2: {day: "Wednesday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Wednesday"
                    open: "00:00"
                3: {day: "Thursday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Thursday"
                    open: "00:00"
                4: {day: "Friday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Friday"
                    open: "00:00"
                5: {day: "Saturday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Saturday"
                    open: "00:00"
                6: {day: "Sunday", open: "00:00", close: "24:00"}
                    close: "24:00"
                    day: "Sunday"
                    open: "00:00"
            departments: []
            distance: 1778.482835104
            facilities: []
            id: 1977
            isChristmasOrderAccepted: false
            isFoodCollectionSupported: false
            isGmCollectionSupported: false
            name: "MOUSEHOLD BP"
            phone: "01603 408611"
            services: []
            specialOpeningHours: []
            storeType: "franchise"
        1: {id: 2642, name: "NORWICH", storeType: "mands", coordinates: {latitude: 52.62612, longitude: 1.29328},…}
        2: {id: 1297, name: "NORWICH SWEET BRIAR SIMPLY FOOD", storeType: "mands",…}
            address: {addressLine1: "NORWICH SWEET BRIAR SIMPLY FOOD", addressLine2: "UNIT 1A SWEET BRIAR NORWICH RETAIL",…}
                addressLine1: "NORWICH SWEET BRIAR SIMPLY FOOD"
                addressLine2: "UNIT 1A SWEET BRIAR NORWICH RETAIL"
                city: "NORWICH"
                country: "United Kingdom"
                county: "NK"
                isoTwoCountryCode: "GB"
                postalCode: "NR6 5DH"
            coordinates: {latitude: 52.651409, longitude: 1.259467}
                latitude: 52.651409
                longitude: 1.259467
            coreOpeningHours: [{day: "Monday", open: "08:00", close: "21:00"}, {day: "Tuesday", open: "08:00", close: "21:00"},…]
                0: {day: "Monday", open: "08:00", close: "21:00"}
                    close: "21:00"
                    day: "Monday"
                    open: "08:00"
                1: {day: "Tuesday", open: "08:00", close: "21:00"}
                    close: "21:00"
                    day: "Tuesday"
                    open: "08:00"
                2: {day: "Wednesday", open: "08:00", close: "21:00"}
                    close: "21:00"
                    day: "Wednesday"
                    open: "08:00"
                3: {day: "Thursday", open: "08:00", close: "21:00"}
                    close: "21:00"
                    day: "Thursday"
                    open: "08:00"
                4: {day: "Friday", open: "08:00", close: "21:00"}
                    close: "21:00"
                    day: "Friday"
                    open: "08:00"
                5: {day: "Saturday", open: "08:00", close: "21:00"}
                    close: "21:00"
                    day: "Saturday"
                    open: "08:00"
                6: {day: "Sunday", open: "10:30", close: "16:30"}
                    close: "16:30"
                    day: "Sunday"
                    open: "10:30"
            departments: [{id: "DPT_BKRY", name: "Bakery"}, {id: "DPT_CARD", name: "Cards & wrap"},…]
                0: {id: "DPT_BKRY", name: "Bakery"}
                    id: "DPT_BKRY"
                    name: "Bakery"
                1: {id: "DPT_CARD", name: "Cards & wrap"}
                    id: "DPT_CARD"
                    name: "Cards & wrap"
                2: {id: "DPT_FWPLNT", name: "Flowers & plants"}
                    id: "DPT_FWPLNT"
                    name: "Flowers & plants"
                3: {id: "DPT_FD", name: "Food"}
                    id: "DPT_FD"
                    name: "Food"
                4: {id: "DPT_FDMV", name: "Food on the move"}
                    id: "DPT_FDMV"
                    name: "Food on the move"
                5: {id: "DPT_FDORDCLLCDSK", name: "Food order collection desk"}
                    id: "DPT_FDORDCLLCDSK"
                    name: "Food order collection desk"
                6: {id: "DPT_WN", name: "Wine"}
                    id: "DPT_WN"
                    name: "Wine"
            distance: 2140.261181148
            facilities: [{id: "FCT_BBYCHNG", name: "Baby changing facilities"}, {id: "FCT_CARPRK", name: "Car parking"},…]
                0: {id: "FCT_BBYCHNG", name: "Baby changing facilities"}
                    id: "FCT_BBYCHNG"
                    name: "Baby changing facilities"
                1: {id: "FCT_CARPRK", name: "Car parking"}
                    id: "FCT_CARPRK"
                    name: "Car parking"
                2: {id: "FCT_TOLT", name: "Toilets"}
                    id: "FCT_TOLT"
                    name: "Toilets"
            id: 1297
            isChristmasOrderAccepted: true
            isFoodCollectionSupported: true
            isGmCollectionSupported: true
            name: "NORWICH SWEET BRIAR SIMPLY FOOD"
            phone: "01603604398"
            services: [{id: "SVC_CLHCD", name: "Clothing & Home collection desk"},…]
                0: {id: "SVC_CLHCD", name: "Clothing & Home collection desk"}
                    id: "SVC_CLHCD"
                    name: "Clothing & Home collection desk"
                1: {id: "SVC_CLHO", name: "Clothing & Home ordering"}
                    id: "SVC_CLHO"
                    name: "Clothing & Home ordering"
                2: {id: "SVC_FPOD", name: "Flower & Plant ordering"}
                    id: "SVC_FPOD"
                    name: "Flower & Plant ordering"
                3: {id: "SVC_FOOD", name: "Food ordering"}
                    id: "SVC_FOOD"
                    name: "Food ordering"
                4: {id: "SVC_WIFI", name: "Free Wi-Fi"}
                    id: "SVC_WIFI"
                    name: "Free Wi-Fi"
                5: {id: "SVC_HPOL", name: "Hamper ordering"}
                    id: "SVC_HPOL"
                    name: "Hamper ordering"
                6: {id: "SVC_CAFE", name: "M&S Cafe"}
                    id: "SVC_CAFE"
                    name: "M&S Cafe"
                7: {id: "SVC_PRCO", name: "Personalised Cake ordering"}
                    id: "SVC_PRCO"
                    name: "Personalised Cake ordering"
                8: {id: "SVC_TAXFRSHP", name: "Tax free shopping"}
                    id: "SVC_TAXFRSHP"
                    name: "Tax free shopping"
                9: {id: "SVC_WCOD", name: "Wedding Cake ordering"}
                    id: "SVC_WCOD"
                    name: "Wedding Cake ordering"
                10: {id: "SVC_WINE", name: "Wine ordering"}
                    id: "SVC_WINE"
                    name: "Wine ordering"
            specialOpeningHours: [{name: "Good Friday", open: "2020-04-10T08:00:00Z", close: "2020-04-10T20:00:00Z", state: "open"},…]
                0: {name: "Good Friday", open: "2020-04-10T08:00:00Z", close: "2020-04-10T20:00:00Z", state: "open"}
                    close: "2020-04-10T20:00:00Z"
                    name: "Good Friday"
                    open: "2020-04-10T08:00:00Z"
                    state: "open"
                1: {name: "Easter Sunday", open: "2020-04-12T00:00:00Z", close: "2020-04-12T00:00:00Z", state: "closed"}
                    close: "2020-04-12T00:00:00Z"
                    name: "Easter Sunday"
                    open: "2020-04-12T00:00:00Z"
                    state: "closed"
                2: {name: "Easter Monday", open: "2020-04-13T08:00:00Z", close: "2020-04-13T18:00:00Z", state: "open"}
                    close: "2020-04-13T18:00:00Z"
                    name: "Easter Monday"
                    open: "2020-04-13T08:00:00Z"
                    state: "open"
                3: {name: "Bank Holiday", open: "2020-05-08T08:00:00Z", close: "2020-05-08T20:00:00Z", state: "open"}
                    close: "2020-05-08T20:00:00Z"
                    name: "Bank Holiday"
                    open: "2020-05-08T08:00:00Z"
                    state: "open"
                4: {name: "Bank Holiday", open: "2020-05-25T08:00:00Z", close: "2020-05-25T20:00:00Z", state: "open"}
                    close: "2020-05-25T20:00:00Z"
                    name: "Bank Holiday"
                    open: "2020-05-25T08:00:00Z"
                    state: "open"
                5: {name: null, open: "2020-08-31T09:00:00Z", close: "2020-08-31T18:00:00Z", state: "open"}
                    close: "2020-08-31T18:00:00Z"
                    name: null
                    open: "2020-08-31T09:00:00Z"
                    state: "open"
            storeType: "mands"
        3: {id: 904, name: "Norwich NHS Hospital", storeType: "mands",…}
        4: {id: 1460, name: "NORWICH LONGWATER FOODHALL", storeType: "mands",…}
        5: {id: 2155, name: "AYLSHAM BP", storeType: "franchise",…}
        6: {id: 1346, name: "ACLE NORWICH BP", storeType: "franchise",…}
        7: {id: 1444, name: "BESTHORPE BP", storeType: "franchise",…}
        8: {id: 1970, name: "LYNN HILL BP", storeType: "franchise",…}
        9: {id: 1412, name: "GREAT YARMOUTH GAPTON HALL FOODHALL", storeType: "mands",…}

# Code for retrieving Marks and Spencer's opening hours data:

In [ ]:
def get_marks_and_spencers_data(lat, lng):
    API_URL = "https://api.marksandspencer.com/v1/stores"
    params = {  'apikey' : "aVCi8dmPbHgHrdCv9gNt6rusFK98VokK",
                'latlong' : "{0},{1}".format(lat,lng),
                'limit' : limit,
                'radius' : radius}
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.text)
        return False
    try:
        res = rq.json()
        if res['count'] == 0:
            return False
        res = res["results"]
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0]["name"])
    #e.g.:
    # while True:
    #     if name != res[i]["name"]:
    #         i += 1
    #     else:
    #         break
    if res[i]["distance"] > radius * 1000:
        return False
    openingHours = res[i]["coreOpeningHours"]
    intermediaryDayKeys = { 'Monday' : 0, 'Tuesday' : 1, 'Wednesday' : 2, 'Thursday' : 3, 'Friday' : 4, 'Saturday' : 5, 'Sunday' : 6}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    dayDoneSet = set()
    daysHeap = []
    for index in range(len(openingHours)):
            key = intermediaryDayKeys[openingHours[index]['day']]  #turn key into int
            #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
            if key not in dayDoneSet:
                dayDoneSet.add(key)
            #keyHours processing code
                try:
                    actualHoursDict = {}
                    actualHoursDict['open'] = openingHours[index]['open']
                    actualHoursDict['close'] = openingHours[index]['close']
                    actualHours = [actualHoursDict]
                    keyHours = {'day' : dayKeys[key],
                        'open' : True,
                        'hours' : actualHours
                        }
                    heapq.heappush(daysHeap, [key,keyHours])
                except:
                    #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                    closedDayHours = {  'day' : dayKeys[key],
                                    'open' : False  }
                    heapq.heappush(daysHeap, [key,closedDayHours])
    # Check for any missing days and ensure all days are in order for insertion into our database
    checkedUpToDay = -1
    for day in heapq.nsmallest(7, daysHeap):
        checkedUpToDay += 1
        while day[0] > checkedUpToDay:
            closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
            heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
            checkedUpToDay += 1
    while len(daysHeap) < 7:
        if len(daysHeap) > 0:
            checkedUpToDay = daysHeap[-1][0] + 1
        else:
            # THIS CHECK MEANS STORE IS OPEN 0 DAYS A WEEK, if we want to remove such stores, insert appropriate code here
            checkedUpToDay = 0
        closedDayHours = {  'day' : dayKeys[checkedUpToDay],
                    'open' : False  }
        heapq.heappush(daysHeap, [checkedUpToDay, closedDayHours])
    if len(daysHeap) != 7:
        return False
    hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
    return hoursArray


# Iceland's API
## Base URL:
https://www.iceland.co.uk/on/demandware.store/Sites-icelandfoodsuk-Site/default/Stores-GetNearestStores
## Parameters:
* latitude = Latitude
* longitude = Longitude
* countryCode = "GB" for UK
* distanceUnit = "mi" for miles
* maxdistance = distance within which to search for stores (measured in distanceUnit)

**Note:** The returned response is not sorted by distance from the given coordinates, this must be done client-side (in this case within the below function) and it is therefore recommended to pass a small "maxdistance" parameter to the API.

## Sample API Call:
https://www.iceland.co.uk/on/demandware.store/Sites-icelandfoodsuk-Site/default/Stores-GetNearestStores?latitude=52.6476395&longitude=1.2932064&countryCode=GB&distanceUnit=mi&maxdistance=50

## Sample Response:

In [ ]:
{stores: {,…}}
stores: {,…}
16: {name: "GORLESTON", storeSecondaryName: "", address1: "138d High Street", address2: "Gorleston On Sea",…}
    address1: "138d High Street"
    address2: "Gorleston On Sea"
    city: "Great Yarmouth"
    countryCode: ""
    email: ""
    fax: ""
    image: ""
    latitude: "52.5782"
    longitude: "1.72711"
    name: "GORLESTON"
    phone: "01493441088"
    postalCode: "NR31 6QX"
    stateCode: ""
    storeEvents: ""
    storeFeatures: "↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵<div class="store-details">↵<div class="store-info-title">Store Details<span>:</span></div>↵<ul>↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Online delivery available↵</li>↵↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Home delivery available↵</li>↵↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Alcohol available↵</li>↵↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Onsite carpark↵</li>↵↵↵</ul>↵</div>↵↵"
    storeHours: "<Store-Hours><div class="mon">Monday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="tue">Tuesday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="wed">Wednesday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="thur">Thursday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="fri">Friday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="sat">Saturday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="Sun">Sunday<div class="store-opening-hours">10:00AM <span>-</span> 4:00PM</div></div></Store-Hours>"
    storeSecondaryName: ""
317: {name: "THETFORD", storeSecondaryName: "", address1: "Unit 1 The Burrells",…}
    address1: "Unit 1 The Burrells"
    address2: "Minstergate Street"
    city: "Thetford"
    countryCode: ""
    email: ""
    fax: ""
    image: ""
    latitude: "52.415"
    longitude: "0.745208"
    name: "THETFORD"
    phone: "01842750746"
    postalCode: "IP24 1BN"
    stateCode: ""
    storeEvents: ""
    storeFeatures: "↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵↵<div class="store-details">↵<div class="store-info-title">Store Details<span>:</span></div>↵<ul>↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Online delivery available↵</li>↵↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Home delivery available↵</li>↵↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Alcohol available↵</li>↵↵↵↵↵↵↵↵↵↵↵↵<li class="default">↵↵Onsite carpark↵</li>↵↵↵</ul>↵</div>↵↵"
    storeHours: "<Store-Hours><div class="mon">Monday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="tue">Tuesday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="wed">Wednesday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="thur">Thursday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="fri">Friday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="sat">Saturday<div class="store-opening-hours">9:00AM <span>-</span> 6:00PM</div></div><div class="Sun">Sunday<div class="store-opening-hours">10:00AM <span>-</span> 4:00PM</div></div></Store-Hours>"
    storeSecondaryName: ""
351: {name: "DEREHAM", storeSecondaryName: "", address1: "10 Nelson Place", address2: "East Dereham",…}
387: {name: "DOWNHAM MARKET", storeSecondaryName: "", address1: "6 Prince of Wales Court",…}
439: {name: "FELIXSTOWE", storeSecondaryName: "", address1: "56/60 Hamilton Road", address2: "Felixstowe",…}
508: {name: "GT YARMOUTH SOUTH RD", storeSecondaryName: "", address1: "90/92 South Rd",…}
588: {name: "NORWICH (2)", storeSecondaryName: "", address1: "Unit 1/3 Anglia Square", address2: "Norwich",…}
752: {name: "LOWESTOFT", storeSecondaryName: "", address1: "15 The Britten Centre", address2: "Lowestoft",…}
768: {name: "NEWMARKET", storeSecondaryName: "", address1: "1 Crown Walk", address2: "Exeter Road",…}
776: {name: "NORWICH (3)", storeSecondaryName: "", address1: "67/73 St Stephens Street",…}
945: {name: "BURY ST EDMUNDS", storeSecondaryName: "", address1: "5/6 Cornhill",…}
992: {name: "ELY", storeSecondaryName: "", address1: "11 The Cloisters", address2: "Brays Lane Ely",…}
1246: {name: "SUDBURY", storeSecondaryName: "", address1: "94 North Street", address2: "Sudbury",…}
1247: {name: "SWAFFHAM", storeSecondaryName: "", address1: "77/81 Market Place", address2: "Swaffham",…}
1263: {name: "DOVERCOURT", storeSecondaryName: "", address1: "246/250 High Street", address2: "Dovercourt",…}
1265: {name: "CROMER", storeSecondaryName: "", address1: "57 Church Street", address2: "Cromer",…}
1377: {name: "IPSWICH TOWER RAMPARTS", storeSecondaryName: "", address1: "Unit 34",…}
1410: {name: "BECCLES", storeSecondaryName: "", address1: "4B/4C Taylors Square", address2: "Newgate",…}
1459: {name: "WISBECH 2", storeSecondaryName: "", address1: "Unit 2 Cromwell Ret Park",…}
1507: {name: "LOWESTOFT FWH", storeSecondaryName: "", address1: "Unit 6B",…}
1516: {name: "GT YARMOUTH FWH", storeSecondaryName: "", address1: "Unit C1",…}
1645: {name: "KINGS LYNN FWH", storeSecondaryName: "", address1: "UNIT A", address2: "HARDWICK RETAIL PARK",…}
1654: {name: "IPSWICH EURO FWH", storeSecondaryName: "", address1: "UNIT 18 EURO RETAIL PARK",…}
1659: {name: "IPSWICH SUFFOLK FWH", storeSecondaryName: "", address1: "SUFFOLK RETAIL PARK",…}
1690: {name: "THETFORD FWH", storeSecondaryName: "", address1: "UNIT D BRECKLAND RET.PK.",…}

# Code for retrieving Iceland's opening hours data:

In [ ]:
def get_iceland_data(lat, lng):
    API_URL = "https://www.iceland.co.uk/on/demandware.store/Sites-icelandfoodsuk-Site/default/Stores-GetNearestStores"
    params = {  'latitude' : lat,
                'longitude' : lng,
                'countryCode' : "GB",
                'distanceUnit' : "km", #NOTE might need to change to "mi" for api to function?
                'maxdistance' : radius}
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()
        if len(res['stores']) == 0:
            return False
        res = res["stores"]
    except:
        return False
    storeHeap = []
    for index in res:
        storeLat, storeLng = float(res[index]["latitude"]), float(res[index]["longitude"])
        #compute distance between the two points using the haversine function
        storeDistance = haversine((lat, lng),(storeLat, storeLng))
        heapq.heappush(storeHeap, [storeDistance,res[index]])
    res = heapq.nsmallest(limit, storeHeap)
    i = 0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0][1]["name"])
    #e.g.:
    # while True:
    #     if name != res[i][1]["name"]:
    #         i += 1
    #     else:
    #         break 
    if res[i][0] > radius:
        return False
    openingHours = res[i][1]["storeHours"]
    cleanr = re.compile('<.*?>|\s|-')
    hoursString = re.sub(cleanr, '', openingHours)
    openingHoursArray = hoursString.split("day")[1:]
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    hoursArray = []
    for index in range(len(openingHoursArray)):
        openingHoursArray[index] = openingHoursArray[index].split("M")[:2]
        for index1 in [0,1]:
            try:
                word = openingHoursArray[index][index1]
                wordSplit = word.split(":")
                if word[-1] == 'A':
                    if wordSplit[0] == "12":
                        word = "00:{0}".format(wordSplit[1][:-1])
                    elif len(wordSplit[0]) == 1:
                        word = "0" + word[:-1]
                    elif len(wordSplit[0]) == 2:
                        word = word[:-1]
                    else: 
                        word = False
                elif word[-1] == 'P':
                    if wordSplit[0] == "12":
                        word = word[:-1]
                    elif len(wordSplit[0]) < 3:
                        word = str( int(wordSplit[0]) + 12) + ":" + wordSplit[1][:-1]
                    else:
                        word = False
                openingHoursArray[index][index1] = word
            except:
                openingHoursArray[index][index1] = False
        if not openingHoursArray[index][0] or not openingHoursArray[index][1]:
            closedDayHours = {  'day' : dayKeys[index],
                                'open' : False  }
            hoursArray.append(closedDayHours)
        else:
            actualHoursDict = {}
            actualHoursDict['open'] = openingHoursArray[index][0]
            actualHoursDict['close'] = openingHoursArray[index][1]
            actualHours = [actualHoursDict]
            keyHours = {'day' : dayKeys[index],
                'open' : True,
                'hours' : actualHours
                }
            hoursArray.append(keyHours)
    return hoursArray

# Kaufland's API
## Base URL: 
https://www.kaufland.de/.klstorefinder.json

## Sample API Call: 
https://www.kaufland.de/.klstorefinder.json

## Sample Response:

In [ ]:
[,…]
[0 … 99]
    0: {n: "DE1310", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.739458", lng: "7.878134",…}
    1: {n: "DE1390", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.320667",…}
        cn: "Kaufland Gevelsberg"
        f: {from: "2019-02-03", to: "2020-10-29"}
        lat: "51.320667"
        lng: "7.339364"
        n: "DE1390"
        p: "02332/92170"
        pc: "58285"
        slf: "CheeseService,SausageService,Barrier-freeWC,Elevators"
        sn: "Großer Markt 7"
        sod: ["2020-08-01|00:00|00:00", "2020-08-02|00:00|00:00", "2020-08-03|00:00|00:00",…]
            0: "2020-08-01|00:00|00:00"
            1: "2020-08-02|00:00|00:00"
            2: "2020-08-03|00:00|00:00"
            3: "2020-08-04|00:00|00:00"
            4: "2020-08-05|00:00|00:00"
            5: "2020-08-06|00:00|00:00"
            6: "2020-08-07|00:00|00:00"
        t: "Gevelsberg"
        wod: ["Monday|07:00|22:00", "Tuesday|07:00|22:00", "Wednesday|07:00|22:00", "Thursday|07:00|22:00",…]
            0: "Monday|07:00|22:00"
            1: "Tuesday|07:00|22:00"
            2: "Wednesday|07:00|22:00"
            3: "Thursday|07:00|22:00"
            4: "Friday|07:00|22:00"
            5: "Saturday|07:00|22:00"
    2: {n: "DE1353", slf: "", lat: "48.724494", lng: "9.352354", cn: "Kaufland Esslingen am Neckar",…}
        cn: "Kaufland Esslingen am Neckar"
        lat: "48.724494"
        lng: "9.352354"
        n: "DE1353"
        p: "0711/9829470"
        pc: "73730"
        slf: ""
        sn: "Alleenstraße 32"
        t: "Esslingen am Neckar OT Zell"
        wod: ["Monday|07:00|22:00", "Tuesday|07:00|22:00", "Wednesday|07:00|22:00", "Thursday|07:00|22:00",…]
            0: "Monday|07:00|22:00"
            1: "Tuesday|07:00|22:00"
            2: "Wednesday|07:00|22:00"
            3: "Thursday|07:00|22:00"
            4: "Friday|07:00|22:00"
            5: "Saturday|07:00|22:00"
    3: {n: "DE1360", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.816106", lng: "12.217098",…}
    4: {n: "DE1443", slf: "CheeseService,SausageService,SnackStand,Barrier-freeWC,Elevators",…}
    5: {n: "DE1323", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "48.737462",…}
    6: {n: "DE1300", slf: "Barrier-freeWC", lat: "52.066022", lng: "6.965075", cn: "Kaufland Ahaus",…}
    7: {n: "DE1303", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "50.999046",…}
    8: {n: "DE1330", slf: "MeatService,CheeseService,SausageService,FishService,Barrier-freeWC",…}
    9: {n: "DE1340", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC", lat: "51.692556",…}
    10: {n: "DE1350",…}
    11: {n: "DE1370", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.251556", lng: "6.327598",…}
    12: {n: "DE1380", slf: "", lat: "51.843504", lng: "6.251638", cn: "Kaufland Emmerich am Rhein",…}
    13: {n: "DE1410", slf: "Barrier-freeWC,Elevators", lat: "51.673095", lng: "6.159703", cn: "Kaufland Goch",…}
    14: {n: "DE1413", slf: "CheeseService,SausageService", lat: "53.793752", lng: "9.980712",…}
    15: {n: "DE1420", slf: "MeatService,CheeseService,SausageService,FishService,Barrier-freeWC",…}
    16: {n: "DE1430", slf: "Barrier-freeWC", lat: "51.665747", lng: "7.828612", cn: "Kaufland Hamm-Mitte",…}
    17: {n: "DE1423", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC", lat: "52.845247",…}
    18: {n: "DE1373",…}
    19: {n: "DE1470", slf: "", lat: "51.695413", lng: "13.261168", cn: "Kaufland Herzberg (Elster)",…}
    20: {n: "DE1520", slf: "", lat: "52.158894", lng: "12.784625", cn: "Kaufland Linthe", p: "033844/75040",…}
    21: {n: "DE1523", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "50.557219", lng: "10.413219",…}
    22: {n: "DE1620", slf: "CheeseService,SausageService", lat: "51.380789", lng: "12.487861",…}
    23: {n: "DE1660", slf: "CheeseService,SausageService", lat: "52.599673", lng: "11.836157",…}
    24: {n: "DE1700", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "52.423064",…}
    25: {n: "DE1720", slf: "CheeseService,SausageService,EVChargingStation_normal,Barrier-freeWC,Elevators",…}
    26: {n: "DE1740", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.966072", lng: "12.085523",…}
    27: {n: "DE1780", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "52.532952",…}
    28: {n: "DE1850", slf: "CheeseService,SausageService,FuelStation,CarPark,Barrier-freeWC,Elevators",…}
    29: {n: "DE1940", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.119260",…}
    30: {n: "DE1960", slf: "CheeseService,SausageService,EVChargingStation_normal,Barrier-freeWC,Elevators",…}
    31: {n: "DE1980",…}
    32: {n: "DE2000", slf: "CheeseService,SausageService,Barrier-freeWC,ParkingLevel,Elevators",…}
    33: {n: "DE2030", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "50.988265", lng: "11.053433",…}
    34: {n: "DE2040", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC,Elevators",…}
    35: {n: "DE2080", slf: "CheeseService,SausageService,FuelStation,Barrier-freeWC", lat: "51.618117",…}
    36: {n: "DE2090",…}
    37: {n: "DE1453", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "49.918632", lng: "10.807095",…}
    38: {n: "DE1463", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "48.684191", lng: "12.692460",…}
    39: {n: "DE1493", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "47.966123", lng: "12.610198",…}
    40: {n: "DE1680",…}
    41: {n: "DE1730", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "48.327128", lng: "9.897468",…}
    42: {n: "DE1810", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "49.749339", lng: "11.538843",…}
    43: {n: "DE1820", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "48.285903", lng: "11.560747",…}
    44: {n: "DE2010",…}
    45: {n: "DE2020",…}
    46: {n: "DE2050", slf: "CheeseService,SausageService,Elevators", lat: "48.128019", lng: "9.802567",…}
    47: {n: "DE2100", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "48.578541",…}
    48: {n: "DE2130", slf: "CheeseService,SausageService,SnackStand", lat: "49.333760", lng: "8.438317",…}
    49: {n: "DE2140",…}
    50: {n: "DE2160",…}
    51: {n: "DE2170", slf: "CheeseService,SausageService,CarPark,Barrier-freeWC", lat: "49.140541",…}
    52: {n: "DE2180", slf: "MeatService,CheeseService,SausageService,FishService,Barrier-freeWC,Elevators",…}
    53: {n: "DE2200", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "48.707676", lng: "8.996997",…}
    54: {n: "DE2210",…}
    55: {n: "DE2220", slf: "Elevators", lat: "49.404656", lng: "8.681305", cn: "Kaufland Heidelberg-Weststadt",…}
    56: {n: "DE2240", slf: "CheeseService,SausageService,FuelStation", lat: "48.713841", lng: "10.161344",…}
    57: {n: "DE2250",…}
    58: {n: "DE1860",…}
    59: {n: "DE1870", slf: "CheeseService,SausageService,SnackStand,Barrier-freeWC", lat: "49.368523",…}
    60: {n: "DE1880", slf: "CheeseService,SausageService,FishService,SnackStand,Barrier-freeWC",…}
    61: {n: "DE1890", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "49.437500", lng: "6.871225",…}
    62: {n: "DE1920", slf: "MeatService,CheeseService,SausageService,FishService,Barrier-freeWC",…}
    63: {n: "DE1450", slf: "CheeseService,SausageService,FuelStation,Barrier-freeWC", lat: "52.216256",…}
    64: {n: "DE1480", slf: "CheeseService,SausageService,FishService", lat: "51.364187", lng: "7.584131",…}
    65: {n: "DE1490", slf: "CheeseService,SausageService", lat: "51.578523", lng: "7.669285",…}
    66: {n: "DE1500", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC", lat: "50.801054",…}
    67: {n: "DE1510", slf: "CheeseService,SausageService", lat: "51.802547", lng: "6.168352",…}
    68: {n: "DE1530", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC", lat: "51.448711",…}
    69: {n: "DE1550", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC,ParkingLevel,Elevators",…}
    70: {n: "DE1560", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.743499", lng: "8.726285",…}
    71: {n: "DE1563", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC,Elevators",…}
    72: {n: "DE1570", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.450688", lng: "7.563034",…}
    73: {n: "DE1580", slf: "CheeseService,SausageService,FuelStation", lat: "50.864133", lng: "8.038579",…}
    74: {n: "DE1583", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.819561", lng: "9.851755",…}
    75: {n: "DE1600", slf: "MeatService,CheeseService,SausageService,FishService,SnackStand,Barrier-freeWC",…}
    76: {n: "DE1630", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.549403",…}
    77: {n: "DE1640", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC", lat: "50.555222",…}
    78: {n: "DE1650", slf: "CheeseService,SausageService", lat: "51.594239", lng: "7.135616",…}
    79: {n: "DE1663", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.715920",…}
    80: {n: "DE1690",…}
    81: {n: "DE1710", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.456500",…}
    82: {n: "DE1750", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.615518",…}
    83: {n: "DE1760", slf: "CheeseService,SausageService", lat: "53.601028", lng: "9.975191",…}
    84: {n: "DE1770", slf: "CheeseService,SausageService,EVChargingStation_normal,Barrier-freeWC,Elevators",…}
    85: {n: "DE1800",…}
    86: {n: "DE1830", slf: "Barrier-freeWC", lat: "52.408623", lng: "9.711369",…}
    87: {n: "DE1840", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "52.181482", lng: "7.854389",…}
    88: {n: "DE1910", slf: "CheeseService,SausageService,CarPark,Barrier-freeWC,Elevators", lat: "49.977415",…}
    89: {n: "DE1930", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "51.654702", lng: "7.096771",…}
    90: {n: "DE1950", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "51.516387",…}
    91: {n: "DE1970",…}
    92: {n: "DE1990",…}
    93: {n: "DE1573", slf: "MeatService,CheeseService,SausageService,Barrier-freeWC,ParkingLevel,Elevators",…}
    94: {n: "DE2083", slf: "CheeseService,SausageService,Barrier-freeWC", lat: "50.588318", lng: "11.813559",…}
    95: {n: "DE1843", slf: "MeatService,CheeseService,SausageService,CarPark", lat: "52.637229",…}
    96: {n: "DE1633", slf: "MeatService,CheeseService,SausageService,FishService,Barrier-freeWC",…}
    97: {n: "DE1693", slf: "CheeseService,SausageService,Barrier-freeWC,Elevators", lat: "49.249192",…}
    98: {n: "DE1713", slf: "MeatService,CheeseService,SausageService,EVChargingStation_normal,Barrier-freeWC",…}
    99: {n: "DE2253", slf: "CheeseService,SausageService", lat: "50.008864", lng: "8.435044",…}
[100 … 199]
[200 … 299]
[300 … 399]
[400 … 499]
[500 … 599]
[600 … 668]

## Code for setting up a database filled with all of Kaufland's opening hours, formatted to our liking:

In [ ]:
def set_up_kaufland_database():
    API_URL = "https://www.kaufland.de/.klstorefinder.json"
    kauflandArray = []
    dayKeys = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday",
    }
    intermediaryDayKeys = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3,
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
        None: None,
    }
    try:
        rq = requests.get(API_URL)
        res = rq.json()
        if len(res) == 0:
            return False
    except:
        return False

    for index in range(len(res)):
        dayDoneSet = set()
        daysHeap = []
        openingHours = res[index]["wod"]
        dayCounter = 0
        for dayIndex in range(len(openingHours)):
            day = dayIndex
            
            #if statement to deal with days being included twice (implemnted due to a bug in rewe system)
            if dayIndex not in dayDoneSet:
                try:
                    dayDoneSet.add(
                        intermediaryDayKeys[openingHours[dayIndex].split("|")[0]]
                    )
                    # keyHours processing code
                    opens = openingHours[dayIndex].split("|")[1]
                    closes = openingHours[dayIndex].split("|")[2]
                    if opens == closes:
                        closedDayHours = {
                            "day": openingHours[dayIndex].split("|")[0],
                            "open": False,
                        }
                        heapq.heappush(daysHeap, [day, closedDayHours])
                    actualHours = {"open": opens, "close": closes}
                    keyHours = {
                        "day": openingHours[dayIndex].split("|")[0],
                        "open": True,
                        "hours": [actualHours],
                    }
                    heapq.heappush(daysHeap, [day, keyHours])

                except:
                    closedDayHours = {
                        "day": openingHours[dayIndex].split("|")[0],
                        "open": False,
                    }
                    heapq.heappush(daysHeap, [day, closedDayHours])

        # Check for any missing days and ensure all days are in order for insertion into our database
        for day in daysHeap:
            if day[1]["day"] != dayKeys[day[0]]:
                day[0] = intermediaryDayKeys[day[1]["day"]]

        if len(daysHeap) < 7:
            for dayDone in dayKeys.keys():
                if dayDone not in dayDoneSet:
                    closedDayHours = {
                        "day": dayKeys[dayDone],
                        "open": False,
                    }
                    heapq.heappush(daysHeap, [dayDone, closedDayHours])

        hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
        latitude = res[index]["lat"]
        longitude = res[index]["lng"]
        kauflandArray.append([(float(latitude), float(longitude)), hoursArray])
    kauflandDB = pd.DataFrame(kauflandArray, columns=["Coordinates", "OpeningHours"])
    kauflandDB.to_csv("Kaufland.csv")
    return True

# Code for retrieving Kaufland's opening hours data from our database (via Haversine):

In [ ]:
def get_kaufland_data(lat, lng):
    # pass lat and lng as arrays of length 1 if you want the storeDistance to be returned too
    if isinstance(lat, list):
        returnDistanceToo = True
        lat = lat[0]
        lng = lng[0]
    else:
        returnDistanceToo = False
    desiredCoords = [lat, lng]
    kauflandDB = pd.read_csv(
        "Kaufland.csv",
        index_col="Unnamed: 0",
        converters={"Coordinates": eval, "OpeningHours": eval},
    )
    kauflandDB["Distances"] = haversine_vector(
        [desiredCoords] * len(kauflandDB["Coordinates"]),
        list(kauflandDB["Coordinates"]),
    )
    closestStoreIndex = kauflandDB["Distances"].idxmin()
    if returnDistanceToo:
        return (
            kauflandDB["Distances"][closestStoreIndex],
            kauflandDB["OpeningHours"][closestStoreIndex],
        )
    else:
        return kauflandDB["OpeningHours"][closestStoreIndex]

# Migros Supermarkt's API

## Base URL:
https://web-api.migros.ch/widgets/stores

## Parameters:
* key = 'loh7Diephiengaiv' (Required api key)
* filters[markets][0][0] = 'super' (Required filter to get Supermarkets)
* filters[markets][0][2] = 'voi' (Required filter to get VOI)
* filters[markets][0][3] = 'mp' (Required filter to get Migros-Partner)
* limit = '737' (Specified to get the number of markets. Given 737 because only got 737 supermarteks by applying filter on the website)


## Sample API Call:
https://web-api.migros.ch/widgets/stores?key=loh7Diephiengaiv&filters[markets][0][0]=super&filters[markets][0][2]=voi&filters[markets][0][3]=mp&limit=737


## Sample API Call Python Code:


In [ ]:
import requests

headers = {
    'Accept-Language': 'de',
    'Origin': 'https://filialen.migros.ch',
}

params = (
    ('key', 'loh7Diephiengaiv'),
    ('filters[markets][0][0]', 'super'),
    ('filters[markets][0][2]', 'voi'),
    ('filters[markets][0][3]', 'mp'),
    ('limit', '736'),
)

response = requests.get('https://web-api.migros.ch/widgets/stores', headers=headers, params=params)
response.json()

## Sample Response:

Below sample response is of only 1 store as to reduce the length of the response.

In [ ]:
{
    "stores": [
        {
            "markets": [
                {
                    "closing_date": null,
                    "media": [],
                    "type": "super",
                    "sap_id": "4762",
                    "logo": {
                        "small": "//image.migros.ch/filialen/50x50/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo.png",
                        "stack": "//image.migros.ch/filialen/{stack}/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo.png",
                        "original": "//image.migros.ch/filialen/original/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo.png",
                        "large": "//image.migros.ch/filialen/600x400/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo.png",
                        "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo.png",
                        "medium": "//image.migros.ch/filialen/200x200/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo.png"
                    },
                    "logo_minimal": {
                        "small": "//image.migros.ch/filialen/50x50/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-minimal.png",
                        "stack": "//image.migros.ch/filialen/{stack}/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-minimal.png",
                        "original": "//image.migros.ch/filialen/original/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-minimal.png",
                        "large": "//image.migros.ch/filialen/600x400/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-minimal.png",
                        "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-minimal.png",
                        "medium": "//image.migros.ch/filialen/200x200/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-minimal.png"
                    },
                    "id": "0090320_super",
                    "slug": "migros-supermarkt-aachtal",
                    "all_types": [
                        "super"
                    ],
                    "email": "fbk-sm-Aachtal@gmos.ch",
                    "opening_date": "2017-02-02",
                    "prefixed_phone": "+41 58 712 59 30",
                    "special_day_schedules": [
                        {
                            "just_message": false,
                            "date_to": "2020-12-25",
                            "time_close1": null,
                            "message": "",
                            "time_close2": null,
                            "holiday": "Weihnachten",
                            "time_open2": null,
                            "time_open1": null,
                            "open": false,
                            "date_from": "2020-12-25"
                        },
                        {
                            "just_message": false,
                            "date_to": "2020-12-26",
                            "time_close1": null,
                            "message": "",
                            "time_close2": null,
                            "holiday": "Stephanstag",
                            "time_open2": null,
                            "time_open1": null,
                            "open": false,
                            "date_from": "2020-12-26"
                        }
                    ],
                    "logo_square": {
                        "small": "//image.migros.ch/filialen/50-sqr/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-square.png",
                        "stack": "//image.migros.ch/filialen/{stack}/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-square.png",
                        "original": "//image.migros.ch/filialen/original-sqr/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-square.png",
                        "large": "//image.migros.ch/filialen/600-sqr/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-square.png",
                        "custom": "//image.migros.ch/filialen/custom-sqr/v-size-{size}/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-square.png",
                        "medium": "//image.migros.ch/filialen/200-sqr/5222e5d31bd3a03018ae0eba14ea06b4aa55b848/super-super-de-logo-square.png"
                    },
                    "assortments": [
                        {
                            "name": "Lebensmittel",
                            "id": "food"
                        }
                    ],
                    "services": [
                        {
                            "sub_type": "subito-selfcheckout",
                            "type": "selfcheckout"
                        },
                        {
                            "sub_type": "subito-selfscanning",
                            "type": "selfscanning"
                        },
                        {
                            "sub_type": "subito",
                            "name": "Self-Scanning / Self-Checkout",
                            "link": "https://www.migros.ch/de/zahlungsmoeglichkeiten/subito.html",
                            "logo": {
                                "small": "//image.migros.ch/filialen/50x50/15cbe934303e781fa4a8107cb962944a24a3ee4e/self-scanning-self-checkout-logo.png",
                                "stack": "//image.migros.ch/filialen/{stack}/15cbe934303e781fa4a8107cb962944a24a3ee4e/self-scanning-self-checkout-logo.png",
                                "original": "//image.migros.ch/filialen/original/15cbe934303e781fa4a8107cb962944a24a3ee4e/self-scanning-self-checkout-logo.png",
                                "large": "//image.migros.ch/filialen/600x400/15cbe934303e781fa4a8107cb962944a24a3ee4e/self-scanning-self-checkout-logo.png",
                                "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/15cbe934303e781fa4a8107cb962944a24a3ee4e/self-scanning-self-checkout-logo.png",
                                "medium": "//image.migros.ch/filialen/200x200/15cbe934303e781fa4a8107cb962944a24a3ee4e/self-scanning-self-checkout-logo.png"
                            },
                            "subito_types": [
                                "selfcheckout",
                                "selfscanning"
                            ],
                            "text": "In dieser Filiale kann man mit Self-Scanning / Self-Checkout selber einscannen, einpacken und danach selbständig und bargeldlos bezahlen.",
                            "type": "subito",
                            "link_text": "Mehr zu Subito Self-Scanning / Self-Checkout"
                        },
                        {
                            "boss_numbers": null,
                            "code": "mig_online",
                            "sub_type": "market-service-mig_online",
                            "name": "Abholort für online Theke",
                            "link": "https://theke.migros.ch/de",
                            "logo": {
                                "small": null,
                                "stack": null,
                                "original": null,
                                "large": null,
                                "custom": null,
                                "medium": null
                            },
                            "text": "Frisch, saisonal und einfach bequem: Bestellen Sie Fleisch online und holen Sie es in Ihrer Wunschfiliale ab.",
                            "type": "market-service",
                            "title": "Abholort für online Theke",
                            "link_text": "Zum Angebot"
                        },
                        {
                            "boss_numbers": null,
                            "code": "mig_bread",
                            "sub_type": "market-service-mig_bread",
                            "name": "Torten Variationen",
                            "link": "",
                            "logo": {
                                "small": null,
                                "stack": null,
                                "original": null,
                                "large": null,
                                "custom": null,
                                "medium": null
                            },
                            "text": "Hier können Sie individuelle dekorierte Torten vorbestellen und abholen.",
                            "type": "market-service",
                            "title": "Torten Variationen",
                            "link_text": ""
                        },
                        {
                            "boss_numbers": null,
                            "code": "mig_gift",
                            "sub_type": "market-service-mig_gift",
                            "name": "Geschenkkörbe",
                            "link": "",
                            "logo": {
                                "small": null,
                                "stack": null,
                                "original": null,
                                "large": null,
                                "custom": null,
                                "medium": null
                            },
                            "text": "Sie lieben Geschenkkörbe? Wir stellen Ihren Wunschkorb mit Sortimentsprodukten zusammen – ganz individuell und für jeden Anlass.",
                            "type": "market-service",
                            "title": "Geschenkkörbe",
                            "link_text": ""
                        },
                        {
                            "sub_type": "srv",
                            "hint": "",
                            "link": "https://www.migros.ch/de/kontakt/ersatzteile-reparaturen.html",
                            "text": "In dieser Filiale können Sie Reparaturen in Auftrag geben, sowie Ersatzteile und Zubehör bestellen und abholen.",
                            "type": "srv",
                            "title": "Reparaturen / Ersatzteile und Zubehör",
                            "link_text": "Mehr zu Reparaturen und Ersatzteile "
                        },
                        {
                            "sub_type": "health-center",
                            "type": "health-center"
                        },
                        {
                            "code": "bargeld_kasse",
                            "sub_type": "bank-service-bargeld_kasse",
                            "name": "Kostenloser Bargeldbezug mit der Maestro-Karte",
                            "link": "",
                            "text": "Wenn Sie Bargeld brauchen, können Sie es an den Bancomaten der Migros Bank abheben. Oder in mehreren hundert Geschäften der Migros Gruppe – unkompliziert und ohne Fremdbankgebühren.",
                            "type": "bank-service",
                            "title": "",
                            "link_text": ""
                        },
                        {
                            "sub_type": "post-service-point",
                            "link": "https://www.migros.ch/de/services/pickmup/zusatzservices-post.html",
                            "logo": {
                                "small": "//image.migros.ch/filialen/50x50/d7e5735451cac67e8d571d49dc112df05eaf1a3f/pakete-abholen-und-aufgeben-logo.png",
                                "stack": "//image.migros.ch/filialen/{stack}/d7e5735451cac67e8d571d49dc112df05eaf1a3f/pakete-abholen-und-aufgeben-logo.png",
                                "original": "//image.migros.ch/filialen/original/d7e5735451cac67e8d571d49dc112df05eaf1a3f/pakete-abholen-und-aufgeben-logo.png",
                                "large": "//image.migros.ch/filialen/600x400/d7e5735451cac67e8d571d49dc112df05eaf1a3f/pakete-abholen-und-aufgeben-logo.png",
                                "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/d7e5735451cac67e8d571d49dc112df05eaf1a3f/pakete-abholen-und-aufgeben-logo.png",
                                "medium": "//image.migros.ch/filialen/200x200/d7e5735451cac67e8d571d49dc112df05eaf1a3f/pakete-abholen-und-aufgeben-logo.png"
                            },
                            "text": "In dieser Filiale können Sie Economy-Pakete versenden und via «PickPost» empfangen.",
                            "type": "post-service-point",
                            "title": "Pakete abholen und aufgeben",
                            "link_text": "SO FUNKTIONIERT DER PAKET-SERVICE"
                        },
                        {
                            "promoted_group": "migros",
                            "pickmup_type": "migrossup",
                            "pickup_location_id": "903",
                            "link": "https://www.pickmup.ch/de.html#so-gehts",
                            "supported_stores": [
                                "ackermann",
                                "amorana",
                                "bikeworld",
                                "brack",
                                "decathlon",
                                "discountle",
                                "doit",
                                "equinet",
                                "exlibris",
                                "fischen",
                                "galaxus",
                                "hauptner",
                                "hauptnerja",
                                "hobbyfarmi",
                                "interio",
                                "jelmoli",
                                "melshop",
                                "micasa",
                                "myluckydog",
                                "orellfuess",
                                "qoqa",
                                "quelle",
                                "quickpac",
                                "reitsport",
                                "sportxx",
                                "yonc"
                            ],
                            "type": "pickmup",
                            "promoted": true,
                            "title": "Dies ist ein PickMup Standort",
                            "return_option": true,
                            "sub_type": "pickmup",
                            "packet_sizes": [
                                "standard"
                            ],
                            "pickmup_type_name": "PickMup Migros",
                            "hint": "Melden Sie sich an der Kasse 1, um Ihre Bestellung abzuholen.",
                            "name": "PickMup",
                            "logo": {
                                "small": "//image.migros.ch/filialen/50x50/505a5c808e744cd82396d8f59790fd73486ef1c0/pickmup-logo.png",
                                "stack": "//image.migros.ch/filialen/{stack}/505a5c808e744cd82396d8f59790fd73486ef1c0/pickmup-logo.png",
                                "original": "//image.migros.ch/filialen/original/505a5c808e744cd82396d8f59790fd73486ef1c0/pickmup-logo.png",
                                "large": "//image.migros.ch/filialen/600x400/505a5c808e744cd82396d8f59790fd73486ef1c0/pickmup-logo.png",
                                "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/505a5c808e744cd82396d8f59790fd73486ef1c0/pickmup-logo.png",
                                "medium": "//image.migros.ch/filialen/200x200/505a5c808e744cd82396d8f59790fd73486ef1c0/pickmup-logo.png"
                            },
                            "text": "Online bestellen und abholen, wann und wo Sie möchten.",
                            "link_text": "So funktioniert PickMup"
                        },
                        {
                            "size": 1,
                            "sub_type": "alna",
                            "logo": {
                                "small": "//image.migros.ch/filialen/50x50/35d1b7e8aefbee61880953372375c3352aa46915/alnatura-sortiment-logo.png",
                                "stack": "//image.migros.ch/filialen/{stack}/35d1b7e8aefbee61880953372375c3352aa46915/alnatura-sortiment-logo.png",
                                "original": "//image.migros.ch/filialen/original/35d1b7e8aefbee61880953372375c3352aa46915/alnatura-sortiment-logo.png",
                                "large": "//image.migros.ch/filialen/600x400/35d1b7e8aefbee61880953372375c3352aa46915/alnatura-sortiment-logo.png",
                                "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/35d1b7e8aefbee61880953372375c3352aa46915/alnatura-sortiment-logo.png",
                                "medium": "//image.migros.ch/filialen/200x200/35d1b7e8aefbee61880953372375c3352aa46915/alnatura-sortiment-logo.png"
                            },
                            "text": "Diese Filiale führt ein Alnatura-Sortiment.",
                            "type": "alna",
                            "title": "Alnatura-Sortiment"
                        }
                    ],
                    "additional_slugs": [],
                    "cooperative": {
                        "name": "Ostschweiz",
                        "id": "gmos",
                        "url": "https://www.migros.ch/de/genossenschaften/migros-ostschweiz.html"
                    },
                    "facebook_id": null,
                    "full_name": "Migros-Supermarkt - Aachtal",
                    "localized_slugs": {
                        "de": "migros-supermarkt-aachtal",
                        "it": "supermercato-migros-aachtal",
                        "fr": "supermarche-migros-aachtal"
                    },
                    "cost_center": "0090320",
                    "phone": "058 712 59 30",
                    "hint": "",
                    "name": "Migros-Supermarkt",
                    "weblink": {
                        "text": "www.migros.ch",
                        "url": "https://www.migros.ch/de.html"
                    },
                    "opening_hours": [
                        {
                            "opening_hours": [
                                {
                                    "time_close1": "20:00",
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": "07:00",
                                    "day_of_week": 1
                                },
                                {
                                    "time_close1": "20:00",
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": "07:00",
                                    "day_of_week": 2
                                },
                                {
                                    "time_close1": "20:00",
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": "07:00",
                                    "day_of_week": 3
                                },
                                {
                                    "time_close1": "20:00",
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": "07:00",
                                    "day_of_week": 4
                                },
                                {
                                    "time_close1": "20:00",
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": "07:00",
                                    "day_of_week": 5
                                },
                                {
                                    "time_close1": "18:00",
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": "07:00",
                                    "day_of_week": 6
                                },
                                {
                                    "time_close1": null,
                                    "time_close2": null,
                                    "time_open2": null,
                                    "time_open1": null,
                                    "day_of_week": 7
                                }
                            ],
                            "active": true,
                            "date_to": null,
                            "date_from": "2019-05-06"
                        }
                    ],
                    "sub_types": [
                        "super"
                    ],
                    "visit_frequencies": null,
                    "managers": [],
                    "computed_opening_hours": []
                }
            ],
            "localized_slugs": {
                "de": "aachtal",
                "it": "aachtal",
                "fr": "aachtal"
            },
            "phone": "058 712 59 30",
            "additional_ids": [
                "0090320_903"
            ],
            "name": "Aachtal",
            "logo": {
                "small": "//image.migros.ch/filialen/50x50/b7ed598472b2aa065658425a494316272885c3ff/m-de.png",
                "stack": "//image.migros.ch/filialen/{stack}/b7ed598472b2aa065658425a494316272885c3ff/m-de.png",
                "original": "//image.migros.ch/filialen/original/b7ed598472b2aa065658425a494316272885c3ff/m-de.png",
                "large": "//image.migros.ch/filialen/600x400/b7ed598472b2aa065658425a494316272885c3ff/m-de.png",
                "custom": "//image.migros.ch/filialen/custom/resize-width-{width}-height-{height}/b7ed598472b2aa065658425a494316272885c3ff/m-de.png",
                "medium": "//image.migros.ch/filialen/200x200/b7ed598472b2aa065658425a494316272885c3ff/m-de.png"
            },
            "location": {
                "geo": {
                    "lon": 9.2732473,
                    "lat": 47.5482982
                },
                "zip": "8580",
                "country": "CH",
                "address": "Schrofenstrasse 19",
                "city": "Amriswil",
                "address2": ""
            },
            "id": "0090320",
            "type": "m",
            "additional_slugs": [],
            "slug": "aachtal",
            "cooperative": {
                "name": "Ostschweiz",
                "id": "gmos",
                "url": "https://www.migros.ch/de/genossenschaften/migros-ostschweiz.html"
            }
        }
    ],
    "total_hits": 737,
    "aggregations": {}
}

## Code for setting up a database filled with all of Migros Supermarkt's opening hours, formatted to our liking:

In [ ]:
def set_up_migros_database():

    API_URL = "https://web-api.migros.ch/widgets/stores"

    headers = {
        "Accept-Language": "de",
        "Origin": "https://filialen.migros.ch",
    }

    params = (
        ("key", "loh7Diephiengaiv"),
        ("filters[markets][0][0]", "super"),
        ("filters[markets][0][2]", "voi"),
        ("filters[markets][0][3]", "mp"),
        ("limit", "737"),
    )

    try:
        rq = requests.get(API_URL, headers=headers, params=params)
        res = rq.json()
        if len(res) == 0:
            return False
    except:
        return False

    migrosArray = []

    dayKeys = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday",
    }
    intermediaryDayKeys = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3,
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
        None: None,
    }

    for index in range(len(res["stores"])):
        dayDoneSet = set()
        daysHeap = []
        openingHours = res["stores"][index]["markets"][0]["opening_hours"][0][
            "opening_hours"
        ]
        dayCounter = 0
        for dayIndex in range(len(openingHours)):
            if dayIndex not in dayDoneSet:
                try:
                    day = openingHours[dayIndex]["day_of_week"] - 1
                    dayDoneSet.add(day)

                    # keyHours processing code
                    # changes done because some stores opens two times a day.
                    opens_firstHalf = openingHours[dayIndex]["time_open1"]
                    closes_firstHalf = openingHours[dayIndex]["time_close1"]
                    opens_secondHalf = openingHours[dayIndex]["time_open2"]
                    closes_secondHalf = openingHours[dayIndex]["time_close2"]
                    if (
                        opens_firstHalf
                        == closes_firstHalf
                        == opens_secondHalf
                        == closes_secondHalf
                    ):
                        closedDayHours = {
                            "day": dayKeys[day],
                            "open": False,
                        }
                        heapq.heappush(daysHeap, [day, closedDayHours])
                    else:
                        if opens_secondHalf == closes_secondHalf:
                            actualHours = {
                                "open": opens_firstHalf,
                                "close": closes_firstHalf,
                            }
                            keyHours = {
                                "day": dayKeys[day],
                                "open": True,
                                "hours": [actualHours],
                            }
                            heapq.heappush(daysHeap, [day, keyHours])
                        else:
                            actualHours_firstHalf = {
                                "open": opens_firstHalf,
                                "close": closes_firstHalf,
                            }
                            actualHours_secondHalf = {
                                "open": opens_secondHalf,
                                "close": closes_secondHalf,
                            }
                            keyHours = {
                                "day": dayKeys[day],
                                "open": True,
                                "hours": [
                                    actualHours_firstHalf,
                                    actualHours_secondHalf,
                                ],
                            }
                            heapq.heappush(daysHeap, [day, keyHours])

                except:
                    closedDayHours = {
                        "day": dayKeys[day],
                        "open": False,
                    }
                    heapq.heappush(daysHeap, [day, closedDayHours])

        # Check for any missing days and ensure all days are in order for insertion into our database
        for day in daysHeap:
            if day[1]["day"] != dayKeys[day[0]]:
                day[0] = intermediaryDayKeys[day[1]["day"]]

        if len(daysHeap) < 7:
            for dayDone in dayKeys.keys():
                if dayDone not in dayDoneSet:
                    closedDayHours = {
                        "day": dayKeys[dayDone],
                        "open": False,
                    }
                    heapq.heappush(daysHeap, [dayDone, closedDayHours])

        hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
        latitude = res["stores"][index]["location"]["geo"]["lat"]
        longitude = res["stores"][index]["location"]["geo"]["lon"]
        migrosArray.append([(float(latitude), float(longitude)), hoursArray])
    migrosDB = pd.DataFrame(migrosArray, columns=["Coordinates", "OpeningHours"])
    migrosDB.to_csv("Migros.csv")
    return True

# Code for retrieving Migros Supermarkt's opening hours data from our database (via Haversine):

In [ ]:
def get_migros_data(lat, lng):
    # pass lat and lng as arrays of length 1 if you want the storeDistance to be returned too
    if isinstance(lat, list):
        returnDistanceToo = True
        lat = lat[0]
        lng = lng[0]
    else:
        returnDistanceToo = False
    desiredCoords = [lat, lng]
    migrosDB = pd.read_csv(
        "Migros.csv",
        index_col="Unnamed: 0",
        converters={"Coordinates": eval, "OpeningHours": eval},
    )
    migrosDB["Distances"] = haversine_vector(
        [desiredCoords] * len(migrosDB["Coordinates"]), list(migrosDB["Coordinates"]),
    )
    closestStoreIndex = migrosDB["Distances"].idxmin()
    if returnDistanceToo:
        return (
            migrosDB["Distances"][closestStoreIndex],
            migrosDB["OpeningHours"][closestStoreIndex],
        )
    else:
        return migrosDB["OpeningHours"][closestStoreIndex]

# Carrefour's Belgium API
## Base URL:
https://magasins.carrefour.eu/api/v3/near/locations/by/slug
## Parameters:
* near = Parameter used to supply coordinates in the following form "radius=Latitude,Longitude". Example: <code>near=51.012676,4.114648</code>
* size = To limit the number of stores returned. <code> size=3 </code>
* radius = Max distance for which results should be shown, measured in Meters (**tested**). Example: <code>radius=20000</code>

## Sample API Call:
https://magasins.carrefour.eu/api/v3/near/locations/by/slug?near=51.012676,4.114648&size=3&radius=20000

## Sample API Call Python Code:

In [ ]:
import requests

params = (
    ('near', '51.012676,4.114648'),
    ('size', '3'),
    ('radius', '20000'),
)

response = requests.get('https://magasins.carrefour.eu/api/v3/near/locations/by/slug', params=params, verify=False)

## Sample Response:

In [ ]:
[
    {
        "guid": "fa166cb5-5626-4eb6-80ee-c65611a2eb1e",
        "siteId": "67f48d7e-5c97-48a4-a474-b67be09dd6c6",
        "externalId": "1498",
        "name": "Carrefour express Dendermonde",
        "status": "OPEN",
        "slug": "carrefour-express-dendermonde",
        "brand": "Carrefour express",
        "brandSlug": "express",
        "language": "fr",
        "description": "",
        "locationItemIds": [
            "service.carrefour.bancontact",
            "service.carrefour.cfrvisa",
            "service.carrefour.itunes",
            "service.carrefour.maestro",
            "service.carrefour.mastercard",
            "service.carrefour.mealvouchers",
            "service.carrefour.nfc",
            "service.carrefour.prepaid",
            "service.carrefour.transport",
            "service.carrefour.visa",
            "service.carrefour.bike.parking",
            "service.carrefour.anti.waste",
            "service.carrefour.biscuits",
            "service.carrefour.bread",
            "service.carrefour.chicken",
            "service.carrefour.flowers",
            "service.carrefour.juice",
            "service.carrefour.localprod",
            "service.carrefour.press",
            "service.carrefour.stamps"
        ],
        "contact": {
            "phone": "052 34 32 94",
            "email": "maximemoonen@hotmail.com",
            "url": "https://winkels.carrefour.eu/nl/s/carrefour/carrefour-express-dendermonde/1498"
        },
        "address": {
            "street": "OUDE VEST, 32",
            "locality": "DENDERMONDE",
            "zipCode": "9200",
            "country": "BE",
            "fullAddress": "OUDE VEST, 32,9200,DENDERMONDE,BE",
            "timezone": "Europe/Brussels",
            "latitude": 51.028634677765645,
            "longitude": 4.1006972184200094,
            "distance": 2.0
        },
        "businessHours": [
            {
                "startDay": 3,
                "endDay": 3,
                "openTime": "0600",
                "closeTime": "2000"
            },
            {
                "startDay": 7,
                "endDay": 7,
                "openTime": "0700",
                "closeTime": "1900"
            },
            {
                "startDay": 6,
                "endDay": 6,
                "openTime": "0700",
                "closeTime": "1900"
            },
            {
                "startDay": 4,
                "endDay": 4,
                "openTime": "0600",
                "closeTime": "2000"
            },
            {
                "startDay": 2,
                "endDay": 2,
                "openTime": "0600",
                "closeTime": "2000"
            },
            {
                "startDay": 5,
                "endDay": 5,
                "openTime": "0600",
                "closeTime": "2000"
            }
        ],
        "specialHours": [],
        "locationExtraData": [],
        "images": [],
        "services": {
            "CAT_FINANCE_JEUX_CARTES": [
                {
                    "externalId": "BANCONTACT",
                    "itemTypeExternalId": "service.carrefour.bancontact",
                    "contentFacet": {
                        "title": "Point de retrait Bancontact",
                        "longContent": "Point de retrait Bancontact"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "CFRVISA",
                    "itemTypeExternalId": "service.carrefour.cfrvisa",
                    "contentFacet": {
                        "title": "Carrefour Visa",
                        "longContent": "Carrefour Visa"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "ITUNES",
                    "itemTypeExternalId": "service.carrefour.itunes",
                    "contentFacet": {
                        "title": "Carte itunes",
                        "longContent": "Carte itunes"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "MAESTRO",
                    "itemTypeExternalId": "service.carrefour.maestro",
                    "contentFacet": {
                        "title": "Maestro",
                        "longContent": "Maestro"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "MASTERCARD",
                    "itemTypeExternalId": "service.carrefour.mastercard",
                    "contentFacet": {
                        "title": "Mastercard",
                        "longContent": "Mastercard"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "MEALVOUCHERS",
                    "itemTypeExternalId": "service.carrefour.mealvouchers",
                    "contentFacet": {
                        "title": "Chèques-repas",
                        "longContent": "Chèques-repas"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "NFC",
                    "itemTypeExternalId": "service.carrefour.nfc",
                    "contentFacet": {
                        "title": "Google Pay",
                        "longContent": "Google Pay"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "PREPAID",
                    "itemTypeExternalId": "service.carrefour.prepaid",
                    "contentFacet": {
                        "title": "Carte téléphonique",
                        "longContent": "Carte  téléphonique"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "TRANSPORT",
                    "itemTypeExternalId": "service.carrefour.transport",
                    "contentFacet": {
                        "title": "Carte de transport",
                        "longContent": "Carte de transport"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                },
                {
                    "externalId": "VISA",
                    "itemTypeExternalId": "service.carrefour.visa",
                    "contentFacet": {
                        "title": "Visa",
                        "longContent": "Visa"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_FINANCE_JEUX_CARTES",
                    "assets": []
                }
            ],
            "CAT_IN_CAR": [
                {
                    "externalId": "BIKE_PARKING",
                    "itemTypeExternalId": "service.carrefour.bike.parking",
                    "contentFacet": {
                        "title": "Parking pour vélo",
                        "longContent": "Parking pour vélo"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_IN_CAR",
                    "assets": []
                }
            ],
            "CAT_SERVICES": [
                {
                    "externalId": "ANTI_WASTE",
                    "itemTypeExternalId": "service.carrefour.anti.waste",
                    "contentFacet": {
                        "title": "Panier anti-gaspillage",
                        "longContent": "Panier anti-gaspillage"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                },
                {
                    "externalId": "BISCUITS",
                    "itemTypeExternalId": "service.carrefour.biscuits",
                    "contentFacet": {
                        "title": "Baguettes/viennoiseries cuites sur place",
                        "longContent": "Baguettes/viennoiseries cuites sur place"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                },
                {
                    "externalId": "BREAD",
                    "itemTypeExternalId": "service.carrefour.bread",
                    "contentFacet": {
                        "title": "Pain cuit sur place",
                        "longContent": "Pain cuit sur place"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                },
                {
                    "externalId": "CHICKEN",
                    "itemTypeExternalId": "service.carrefour.chicken",
                    "contentFacet": {
                        "title": "Poulet rôti",
                        "longContent": "Poulet rôti"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                },
                {
                    "externalId": "FLOWERS",
                    "itemTypeExternalId": "service.carrefour.flowers",
                    "contentFacet": {
                        "title": "Fleurs",
                        "longContent": "Fleurs"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                },
                {
                    "externalId": "JUICE",
                    "itemTypeExternalId": "service.carrefour.juice",
                    "contentFacet": {
                        "title": "Jus d'orange frais",
                        "longContent": "Jus d'orange frais"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                },
                {
                    "externalId": "LOCALPROD",
                    "itemTypeExternalId": "service.carrefour.localprod",
                    "contentFacet": {
                        "title": "Produits locaux",
                        "longContent": "Produits locaux"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_SERVICES",
                    "assets": []
                }
            ],
            "CAT_TELEPHONIE_POSTE_PAPIER": [
                {
                    "externalId": "PRESS",
                    "itemTypeExternalId": "service.carrefour.press",
                    "contentFacet": {
                        "title": "Coin presse",
                        "longContent": "Coin presse"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_TELEPHONIE_POSTE_PAPIER",
                    "assets": []
                },
                {
                    "externalId": "STAMPS",
                    "itemTypeExternalId": "service.carrefour.stamps",
                    "contentFacet": {
                        "title": "Vente de timbres",
                        "longContent": "Vente de timbres"
                    },
                    "validityFacet": [],
                    "priceFacet": [],
                    "callToActionFacet": [],
                    "itemTypeCategoryLabel": "CAT_TELEPHONIE_POSTE_PAPIER",
                    "assets": []
                }
            ]
        }
    }
]

# Code for retrieving Carrefour's Belgium opening hours data:

In [ ]:
def get_carrefour_data(lat, lng):
    API_URL = "https://magasins.carrefour.eu/api/v3/near/locations/by/slug"
    params = (
        ("near", "{},{}".format(lat, lng)),
        ("size", "{}".format(limit)),
        ("radius", "20000"),
    )
    rq = requests.get(API_URL, params=params, verify=False)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()
        if len(res) == 0:
            return False
    except:
        return False
    i = 0
    # INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0]["name"])
    # e.g.:
    # while True:
    #     if name != res[i]["name"]:
    #         i += 1
    #     else:
    #         break
    storeLat, storeLng = (
        float(res[i]["address"]["latitude"]),
        float(res[i]["address"]["longitude"]),
    )
    # compute distance between the two points using the haversine function
    storeDistance = haversine((lat, lng), (storeLat, storeLng), unit="m")
    if storeDistance > radius:
        return False
    openingHours = res[i]["businessHours"]

    carrefourArray = []
    dayKeys = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday",
    }
    intermediaryDayKeys = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3,
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
        None: None,
    }

    for index in range(len(res)):
        dayDoneSet = set()
        daysHeap = []
        openingHours = res[index]["businessHours"]
        dayCounter = 0
        for dayIndex in range(len(openingHours)):
            day = dayIndex
            dayDone = openingHours[dayIndex]["startDay"] - 1
            # if statement to deal with days being included twice (implemnted due to a bug in rewe system)
            if dayDone not in dayDoneSet:
                try:
                    dayDoneSet.add(dayDone)
                    # keyHours processing code
                    opens = openingHours[dayIndex]["openTime"]
                    closes = openingHours[dayIndex]["closeTime"]
                    if opens == closes:
                        closedDayHours = {
                            "day": dayKeys[dayDone],
                            "open": False,
                        }
                        heapq.heappush(daysHeap, [day, closedDayHours])
                    actualHours = {"open": opens, "close": closes}
                    keyHours = {
                        "day": dayKeys[dayDone],
                        "open": True,
                        "hours": [actualHours],
                    }
                    heapq.heappush(daysHeap, [day, keyHours])

                except:
                    closedDayHours = {
                        "day": dayKeys[dayDone],
                        "open": False,
                    }
                    heapq.heappush(daysHeap, [day, closedDayHours])

        # Check for any missing days and ensure all days are in order for insertion into our database
        for day in daysHeap:
            if day[1]["day"] != dayKeys[day[0]]:
                day[0] = intermediaryDayKeys[day[1]["day"]]

        if len(daysHeap) < 7:
            for dayDone in dayKeys.keys():
                if dayDone not in dayDoneSet:
                    closedDayHours = {
                        "day": dayKeys[dayDone],
                        "open": False,
                    }
                    heapq.heappush(daysHeap, [dayDone, closedDayHours])

        hoursArray = [i[1] for i in heapq.nsmallest(7, daysHeap)]
        latitude = res[index]["address"]["latitude"]
        longitude = res[index]["address"]["longitude"]
        carrefourArray.append([(float(latitude), float(longitude)), hoursArray])
    return carrefourArray

# X's API
## Base URL:

## Parameters:


## Sample API Call:

## Sample Response:

# Code for retrieving X's opening hours data:

# Y's API (Germany)
## Base URL:

## Parameters:


## Sample API Call:

## Sample Response:

# Code for retrieving Y's opening hours data:

# Z's API (Germany)
## Base URL:

## Parameters:


## Sample API Call:

## Sample Response:

# Code for retrieving Z's opening hours data: